<a href="https://colab.research.google.com/github/mille-s/Build_KGs_entities/blob/main/DBpedia_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Clones, Installs and functions
from IPython.display import clear_output, HTML, display
import os
! pip install SPARQLWrapper
! pip install colored
! pip install xmltodict

# Clone Build_KGs_entities repo
! git clone https://github.com/mille-s/Build_KGs_entities.git
# Delete locally to avoid confusion
! rm '/content/Build_KGs_entities/DBpedia_dataset.ipynb'

# clone wikipedia page generator repo
! git clone https://github.com/mille-s/WikipediaPage_Generator.git
# Delete locally to avoid confusion
! rm 'WikipediaPage_Generator/Wikipedia_generator.ipynb'

# clone dcu_tcd_webnlg repo
! git clone https://github.com/mille-s/DCU_TCD_FORGe_WebNLG23.git
# Delete locally to avoid confusion
! rm 'DCU_TCD_FORGe_WebNLG23/DCU_TCD_FORGe_WebNLG23.ipynb'

triple2predArg = os.path.join('/content', 'XML')
os.makedirs(triple2predArg)

def aggregate_info_and_get_propLabel(ontology_properties):
  dico_properties = {}
  for prop in ontology_properties:
    # Get raw labels without the url part
    _ , property_no_prefix = os.path.split(prop['property'])
    domain_no_prefix = 'Unknown'
    if not prop['domain'] == 'Unknown':
      _ , domain_no_prefix = os.path.split(prop['domain'])
    range_no_prefix = 'Unknown'
    if not prop['range'] == 'Unknown':
      _ , range_no_prefix = os.path.split(prop['range'])
    # The first time a property is found, create a dico entry with domain and range info
    if property_no_prefix not in dico_properties.keys():
      dico_properties[property_no_prefix] = {'domain': [domain_no_prefix], 'range': [range_no_prefix]}
    # The second time, only append the domain and range if they haven't been seen to this point
    else:
      if domain_no_prefix not in dico_properties[property_no_prefix]['domain']:
        dico_properties[property_no_prefix]['domain'].append(domain_no_prefix)
      if range_no_prefix not in dico_properties[property_no_prefix]['range']:
        dico_properties[property_no_prefix]['range'].append(range_no_prefix)
  return dico_properties

clear_output()
print('Working folder ready!\n--------------')

Working folder ready!
--------------


# Preliminary work: get info to build datasets

In [ ]:
#@title SPARQL query to get all properties in DBpedia (from ChatGPT)

from SPARQLWrapper import SPARQLWrapper, JSON

def getDBpediaProperties():
  # Set up the SPARQL endpoint
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  # Define the SPARQL query to retrieve properties with the prefix "http://dbpedia.org/ontology"
  query = """
  SELECT DISTINCT ?property
  WHERE {
    ?property a rdf:Property .
    FILTER(STRSTARTS(STR(?property), "http://dbpedia.org/ontology/"))
  }
  ORDER BY ?property
  """

  # Set the query
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  try:
      # Execute the query
      results = sparql.query().convert()

      # Process and display the results
      properties = [result["property"]["value"] for result in results["results"]["bindings"]]
      print(f"Number of properties used in DBpedia: {str(len(properties))}")
      # for prop in properties:
      #     print(prop)

  except Exception as e:
      print(f"An error occurred: {e}")

  return properties

list_properties = getDBpediaProperties()

In [ ]:
#@title SPARQL query to get the number of instances of each property
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import json

def getNumInstancesProperty(property_label):
  # lowercase first character
  head, tail = os.path.split(property_label)
  lowCase_tail = tail[0].lower() + tail[1:]
  lowCase_property_label = os.path.join(head, lowCase_tail)

  # Set up the SPARQL endpoint
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  # Define the SPARQL query to count the number of instances of the property 'dbo:birthDate'
  query = f"""
  SELECT (COUNT(*) AS ?count)
  WHERE {{
    ?subject <{lowCase_property_label}> ?object .
  }}
  """

  # Set the query
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  try:
      # Execute the query
      results = sparql.query().convert()

      # Extract and print the count
      count = results["results"]["bindings"][0]["count"]["value"]
      # print(f"Number of instances of {lowCase_property_label}: {count}")

  except Exception as e:
      print(f"An error occurred: {e}")

  return(lowCase_property_label, count)

def createDicoCountOccurrenceProperties(list_properties):
  dico_count_occurrences = {}
  for i, property_label in enumerate(list_properties):
    lowCase_property_label, count = getNumInstancesProperty(property_label)
    dico_count_occurrences[lowCase_property_label] = int(count)
    print(f'{str(i)}/{str(len(list_properties))}: {property_label} = {count}')

  sorted_dico_count_occurrences = {k: v for k, v in sorted(dico_count_occurrences.items(), key=lambda item: item[1], reverse=True)}
  with open("dico_count_occurrences_dbp_props.json", "w") as outfile:
      json.dump(sorted_dico_count_occurrences, outfile)

createDicoCountOccurrenceProperties(list_properties)

In [ ]:
#@title SPARQL query for all properties getting domain/range class
from SPARQLWrapper import SPARQLWrapper, JSON

# Set up the DBpedia SPARQL endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

# SPARQL query to select ontology properties with domain and range
# This is supposed to return all properties, but a lot seem to be missing, not sure why
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?property ?domain ?range WHERE {
    ?property a rdf:Property .
    FILTER(STRSTARTS(STR(?property), "http://dbpedia.org/ontology/")) .
    OPTIONAL { ?property rdfs:domain ?domain . }
    OPTIONAL { ?property rdfs:range ?range . }
}
LIMIT 50000  # Increase this limit if needed
"""

# Run the query
sparql.setQuery(query)
results = sparql.query().convert()

# Extract and display the results
ontology_properties = []
for result in results["results"]["bindings"]:
    property_uri = result["property"]["value"]
    domain = result.get("domain", {}).get("value", "Unknown")
    range_class = result.get("range", {}).get("value", "Unknown")
    ontology_properties.append({
        "property": property_uri,
        "domain": domain,
        "range": range_class
    })

# Print or process the results
for prop in ontology_properties[:100]:
    print(f"Property: {prop['property']}")
    print(f"  Domain: {prop['domain']}")
    print(f"  Range: {prop['range']}")
    print()




In [ ]:
#@title SPARQL query for selected properties getting domain/range class
from SPARQLWrapper import SPARQLWrapper, JSON

WebNLG_properties_list = ['http://dbpedia.org/ontology/'+line.strip() for line in codecs.open('/content/all_properties.txt', 'r', 'utf-8').readlines()]

def get_domain_range(properties):
    domain_range_info = []

    for property_uri in properties:
        print(f'Cheking property {property_uri}')
        # SPARQL query to get domain and range for the specific property
        query = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?domain ?range WHERE {{
            <{property_uri}> a rdf:Property .
            OPTIONAL {{ <{property_uri}> rdfs:domain ?domain . }}
            OPTIONAL {{ <{property_uri}> rdfs:range ?range . }}
        }}
        """

        # Run the query
        sparql.setQuery(query)
        results = sparql.query().convert()

        # Extract domain and range information from the results
        for result in results["results"]["bindings"]:
            domain = result.get("domain", {}).get("value", "Unknown")
            range_class = result.get("range", {}).get("value", "Unknown")
            domain_range_info.append({
                "property": property_uri,
                "domain": domain,
                "range": range_class
            })

    return domain_range_info

# Retrieve domain and range for each property in the list
ontology_properties = get_domain_range(WebNLG_properties_list)

In [ ]:
#@title Aggregate possible domain/ranges for each property
import os
import codecs

dico_properties = aggregate_info_and_get_propLabel(ontology_properties)

# for prop in properties_info:
#     print(f"Property: {prop['property']}", f"  Domain: {prop['domain']}", f"  Range: {prop['range']}")

# print(len(dico_properties.keys()))
# print(len(properties_info))

list_properties = [line.strip() for line in codecs.open('/content/all_properties.txt', 'r', 'utf-8').readlines()]
missing_props = []
for WebNLG_property in list_properties:
  if WebNLG_property not in dico_properties.keys():
    missing_props.append(WebNLG_property)

print('Missing properties: '+str(len(missing_props))+'/'+str(len(list_properties)), missing_props)
# We need to check the Original property labels, not the modified ones, that way we should get them all


In [ ]:
#@title Get examples for properties
# Get a list of sample subject-object values given an input list of properties.

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import csv
import json

num_examples_desired = 30#@param

def get_dbpedia_property_examples(property_uri, dataframe_examples, count_props, num_occ_uri, limit=10):
  sparql = SPARQLWrapper("https://dbpedia.org/sparql")
  query = f"""
  SELECT DISTINCT ?subject ?object WHERE {{
    ?subject <{property_uri}> ?object .
  }} LIMIT {limit}
  """
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  count_examples = 0
  for result in results["results"]["bindings"]:
    subject_uri = result["subject"]["value"]
    object_value = result["object"]["value"]

    # Extract a readable name from the subject and object URI
    subject_label = ""
    if subject_uri.startswith("http://dbpedia.org/resource/"):
      subject_label = subject_uri.split("/")[-1].replace("_", " ")
    else:
      subject_label = subject_uri
    object_label = ""
    if object_value.startswith("http://dbpedia.org/resource/"):
      object_label = object_value.split("/")[-1].replace("_", " ")
    else:
      object_label = object_value

    prop_label = property_uri.split("/")[-1]  # e.g., 'birthDate'
    dataframe_examples.loc[count_props*limit+count_examples] = [count_props, num_occ_uri, subject_label, prop_label, object_label]
    count_examples += 1

# properties_uri = ["http://dbpedia.org/ontology/birthDate", "http://dbpedia.org/ontology/birthPlace"]
list_props_dico = json.load(open('/content/dico_count_occurrences_dbp_props.json', 'r'))
# Get list of URIs for which there is at least 10 occurrences of the property (to filter out possibly bad properties)
properties_uri = []
# Also get list of the number of occurrences of each URI to store in the final CSV
num_occ_uris = []
for prop in list_props_dico.keys():
  if list_props_dico[prop] >= 10:
    properties_uri.append(prop)
    num_occ_uris.append(list_props_dico[prop])
assert len(properties_uri) == len(num_occ_uris)
print(f'Number of properties with at least 10 occurrences: {len(properties_uri)}')

# Create dataframe
dataframe_examples = pd.DataFrame(columns=["id", "num-occurrences", "Subject", "Property", "Object"])
# Example usage:
for counter_uris, property_uri in enumerate(properties_uri):
  print(f'{str(counter_uris)}/{str(len(properties_uri))}: {property_uri}...')
  num_occ_uri = num_occ_uris[counter_uris]
  get_dbpedia_property_examples(property_uri, dataframe_examples, counter_uris, num_occ_uri, int(num_examples_desired))

# Save dataframe as CSV
dataframe_examples.to_csv('dbp_props_examples.csv', index=False)

print(dataframe_examples)

In [ ]:
#@title Process CSV created from annotated examples for properties and get lists of properties to ignore and that can happen only once.
# The produced files can be found in the GitHub WikipediaPage_Generator repo.
# I needed to extract which properties can happen multiple times for a given subject and which can't, as annotated manually in a spreadsheet.
# Guidelines for annotating a property are "is it confusing if ever a text contains two values for that property". E.g. "X was born in Finland and Britain" is weird, but "X is the sister of Y and Z" is not.
# It doesn't necessarily have to do with factual truth, e.g. it is almost certain that a ID for an entity in a database is unique, but it's not shocking to say "X has IDs A and B in database W".
# On the other hand, some cases with multiple object values are annotated as having a single value to avoid poorly entered DBpedia values: e.g. locations (frequently, people put as 3 values a town, the state, the country), subclasses/types (one value is more natural in a text), etc.

import pandas as pd
import csv
import json

# Load the CSV file into a DataFrame
# CSV file in C:\Users\sfmil\Desktop\ADAPT-2025-2026\MyPapers\2025-06_INLG-longText-D2T
df = pd.read_csv('/content/dbp_props_examples_annotated.csv')

# Make a new dataframe with the first 1000 rows of df
# df_1000 = df.head(1000)

list_props_that_can_happen_once_only = []
list_props_that_can_happen_more_than_once = []
list_props_to_filter = []
# Iterate over the rows of the dataframe
current_property = None
for index, df_row in df.iterrows():
  if df_row['Property'] != current_property:
    current_property = df_row['Property']
    current_num_possible_values = df_row['num-possible\nvalues']
    current_id = df_row['id']
    # print(f"Index: {index} - Subject: {df_row['Subject']} - Property: {df_row['Property']} - Object: {df_row['Object']}")
    if current_num_possible_values == '1':
      list_props_that_can_happen_once_only.append(current_property)
      # print(f'ID: {str(current_id).zfill(4)} - Status: {current_num_possible_values} {current_property}')
    elif current_num_possible_values == 'Multiple':
      list_props_that_can_happen_more_than_once.append(current_property)
      # print(f'ID: {str(current_id).zfill(4)} - Status: M {current_property}')
    # else is "?" or N/A
    elif current_num_possible_values == '?' or current_num_possible_values == 'IgnoreProp':
      list_props_to_filter.append(current_property)
      # print(f'ID: {str(current_id).zfill(4)} - Status: - {current_property}')
    else:
      print('ERROR! Unexpected value')
      break

print(sorted(list_props_that_can_happen_once_only))
print(len(list_props_that_can_happen_once_only))
print()
print(sorted(list_props_that_can_happen_more_than_once))
print(len(list_props_that_can_happen_more_than_once))
print()
print(sorted(list_props_to_filter))
print(len(list_props_to_filter))

print(f'There are {len(list_props_that_can_happen_once_only)+len(list_props_that_can_happen_more_than_once)+len(list_props_to_filter)} properties collected (expected: 1208).')

# Save lists as json
with open("list_props_that_can_happen_once_only.json", "w") as outfile:
    json.dump(sorted(list_props_that_can_happen_once_only), outfile)
# with open("list_props_that_can_happen_more_than_once.json", "w") as outfile:
#     json.dump(list_props_that_can_happen_more_than_once, outfile)
with open("list_props_to_filter.json", "w") as outfile:
    json.dump(sorted(list_props_to_filter), outfile)

In [ ]:
#@title SPARQL query for finding all the possible values for gold:hypernym on dbpedia
# Could use dbo:type or rdf:type, but both look a bit messy at first sight
from SPARQLWrapper import SPARQLWrapper, JSON

def get_types():
  # Define the SPARQL endpoint
  sparql = SPARQLWrapper("https://dbpedia.org/sparql")

  # Define the SPARQL query
  query = """
  SELECT DISTINCT ?type
  WHERE {
    ?s gold:hypernym ?type .
  }
  """

  # Set the query and the return format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and retrieve results
  # Returns 10k results, and running it several times in a row always returns the same 10k results
  results = sparql.query().convert()

  return results["results"]["bindings"]

def count_entities_of_type(hypernym_type):
  # Define the DBpedia SPARQL endpoint
  sparql = SPARQLWrapper("https://dbpedia.org/sparql")

  # Define the SPARQL query
  query = f"""
  SELECT (COUNT(?s) AS ?count)
  WHERE {{
      ?s gold:hypernym <{hypernym_type}> .
  }}
  """

  # Set the query and the return format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and retrieve results
  results = sparql.query().convert()

  # Extract and return the count
  count = results["results"]["bindings"][0]["count"]["value"]
  return int(count)

dico_hypernym_types = {}
# Extract and print the types
results_types = get_types()
for i, result in enumerate(results_types):
  hypernym_url = result['type']['value']
  if hypernym_url not in dico_hypernym_types.keys():
    count_occurrences = count_entities_of_type(hypernym_url)
    dico_hypernym_types[hypernym_url] = count_occurrences
    print(f'{str(i)}/{str(len(results_types))}: {hypernym_url} = {count_occurrences}')


In [ ]:
#@title Save dico_hypernym_types to a json and download
import json
from google.colab import files

with open("dico_hypernym_types.json", "w") as outfile:
  sorted_dico_hypernym_types = {k: v for k, v in sorted(dico_hypernym_types.items(), key=lambda item: item[1], reverse=True)}

  json.dump(sorted_dico_hypernym_types, outfile)

# Download
files.download('dico_hypernym_types.json')

In [ ]:
#@title Make a list of random entities to query for each hypernym (SPARQL query for getting n entities that have a specific hypernym)
import random
import json
from SPARQLWrapper import SPARQLWrapper, JSON

def get_random_entities(hypernym: str, limit):
  sparql = SPARQLWrapper("http://dbpedia.org/sparql")

  query = f"""
  SELECT DISTINCT ?entity WHERE {{
      ?entity gold:hypernym <{hypernym}> .
  }} LIMIT 50000
  """

  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  entities = [result['entity']['value'] for result in results['results']['bindings']]
  return random.sample(entities, min(len(entities), limit))

# Load json that contains hypernyms as keys and count of instances of that hypernym on DBpedia as value
dico_hypernym_types = None
with open('/content/dico_hypernym_types_incomplete.json', 'r') as file:
    dico_hypernym_types = json.load(file)

# Get up to 1000 random entities for the classes that have at least 100 members
dico_hypernym_sample_entities = {}
for i, hypernym in enumerate(dico_hypernym_types.keys()):
  if i < 5:
    if dico_hypernym_types[hypernym] >= 100:
      print(hypernym, dico_hypernym_types[hypernym])
      dico_hypernym_sample_entities[hypernym] = get_random_entities(hypernym, 10)

In [ ]:
#@title Save dico_hypernym_sample_entities as JSON and download
from google.colab import files
with open("dico_hypernym_sample_entities.json", "w") as outfile:
    json.dump(dico_hypernym_sample_entities, outfile)
# Dowload json
files.download('dico_hypernym_sample_entities.json')

# Build dataset

## Get properties for list of entities
Creates dico_input_contents_DBp.pickle file.
Skip if you want to use an already generated pickle file.

Triple validation takes a lot of time currently, so only activate here if you don't have a large amount of triples to check.

In [ ]:
# @title Get DBpedia properties online for an entity list (about 1h for GREC entities)
import os
import codecs
import json
import re
import ipywidgets as widgets
from ipywidgets import Layout
from WikipediaPage_Generator.code.queryDBpediaProps import get_dbpedia_properties
from WikipediaPage_Generator.code.utils import removeReservedCharsFileName

# Input json should be a dico_1 with category names (urls or name) as keys, and a list of entities (urls or names) as value.
input_json_path = '/content/random_output.json'#@param{type:"string"}
# triple-source should be Ontology for this experiment
triple_source = 'Ontology' #@param['Infobox', 'Ontology', 'Wikidata']
# (Sub)set of properties to consider (about 400 in WebNLG, about 1.2K in DBpedia all)
consider_properties = 'DBpedia_all' #@param['DBpedia_WebNLG20', 'DBpedia_all']
# Store here "dirty" properties; string expected, it is later on split by ','
# ignore_properties = 'width,title' # Used for semantic accuracy experiments with Rudali
ignore_properties = ','.join(json.loads(open('/content/WikipediaPage_Generator/resources/list_props_to_filter.json', 'r').read()))
get_triples_where_entity_is_subj = True #@param {type:"boolean"}
get_triples_where_entity_is_obj = True #@param {type:"boolean"}
triple_validation = False #@param {type:"boolean"}
props_list_path = ''
if consider_properties == 'DBpedia_WebNLG20':
  props_list_path = os.path.join('/content', 'DCU_TCD_FORGe_WebNLG23', 'code', 'sorted_properties.txt')
elif consider_properties == 'DBpedia_all':
  props_list_path = os.path.join('/content', 'WikipediaPage_Generator', 'resources', 'list_DBpedia_props_seenOver10times.txt')

# Load json dico with sample entities for each hypernym
dico_hypernym_sample_entities_loaded = None
with open(input_json_path, 'r') as file:
    dico_hypernym_sample_entities_loaded = json.load(file)

# dico_input_contents will contain category keys, which contain entity keys, which contain a list of triple objects
dico_input_contents = {}
for hypernym in sorted(dico_hypernym_sample_entities_loaded.keys()):
  input_category = None
  if re.search('/', hypernym):
    input_category = hypernym.rsplit('/', 1)[1]
  else:
    input_category = hypernym
  print(input_category)
  dico_input_contents[input_category] = {}
  # Format properties for passing as argument to python module
  # list_triple_object contains object with 3 attributes: DBsubj, DBprop, DBobj
  # list_propObj is used for UI (for triples selection by the user)
  # list_obj is used for getting class and gender info later on
  ##############################################################################
  # WARNING-TODO: I forgot to apply removeReservedCharsFileName to entity_names and replace spaces by underscores
  ##############################################################################
  for sampled_entity in sorted(dico_hypernym_sample_entities_loaded[hypernym]):
    entity_name = None
    if re.search('/', sampled_entity):
      entity_name = sampled_entity.rsplit('/', 1)[1]
    else:
      entity_name = '_'.join(sampled_entity.split(' '))

    # Get all triples in which the entity is the subject
    list_triple_objects, list_propObj, list_obj = get_dbpedia_properties(props_list_path, entity_name, triple_source, ignore_properties, get_triples_where_entity_is_subj, get_triples_where_entity_is_obj, triple_validation)

    if len(list_triple_objects) > 0:
      print(f'  {entity_name}: found {len(list_triple_objects)} properties.')
      dico_input_contents[input_category][entity_name] = list_triple_objects

geography
  Addis_Ababa: found 625 properties.
  Adriatic_Sea: found 281 properties.
  Afghanistan: found 2968 properties.
  Algeria: found 4488 properties.
  Amur: found 31 properties.
  Angola: found 1543 properties.
  Apennine_Mountains: found 135 properties.
  Arabian_Sea: found 242 properties.
  Arctic_Circle: found 2 properties.
  Armenia: found 2832 properties.
  Athens: found 3113 properties.
  Atlanta: found 4106 properties.
  Atlantic_Ocean: found 1458 properties.
  Atlas: found 15 properties.
  Atlas_Mountains: found 24 properties.
  Aïr_Mountains: found 11 properties.
  Baffin_Bay: found 56 properties.
  Balkans: found 149 properties.
  Baltic_Sea: found 353 properties.
  Bay_of_Biscay: found 67 properties.
  Beaufort_Sea: found 36 properties.
  Belgrade: found 4217 properties.
  Berlin: found 6270 properties.
  Bermuda: found 538 properties.
  Black_Sea: found 199 properties.
  Bogotá: found 1812 properties.
  Borneo: found 116 properties.
  Bosnia_and_Herzegovina: found 6

In [ ]:
#@title Serialise dico_input_contents using pickle and download
import pickle
from google.colab import files

with open("dico_input_contents_DBp.pickle", "wb") as handle:
    pickle.dump(dico_input_contents, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Download file
files.download('dico_input_contents_DBp.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Build knowledge graphs #1: WebNLG mirror input configuration

In [ ]:
#@title Check which entities have property (sub)sets that match WebNLG inputs. Creates dico_entities_for_triple_configuration.json file.
import json
import pickle

dico_input_contents = 'GitHub_GREC_NEs'#@param['GitHub_GREC_NEs', 'Made_with_this_notebook']
print_output = False #@param{type:'boolean'}

# dico_input_contents_loaded should be a dico_1 with category names as keys, and a dico_2 as value.
# dico_2 should have entity_name as key, and a list of Triple Objects as value.
# Triple Objects have 3 attributes: DBsubj, DBprop, DBobj
dico_input_contents_loaded = None
if dico_input_contents == 'Made_with_this_notebook':
  with open("dico_input_contents_DBp.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)
elif dico_input_contents == 'GitHub_GREC_NEs':
  with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)

# dico_triple_configs_WebNLG should be a dico_1 with category names as keys, and a dico_2 as value.
# dico_2 should have strings of properties separated by "##" as keys, and integers (occurrence counts in WebNLG train data) as values.
dico_triple_configs_WebNLG = None
with open("/content/Build_KGs_entities/resources/dico_category_tripleConfigs_WebNLG.json", "r") as handle:
    dico_triple_configs_WebNLG = json.load(handle)

# dico_mapping_categories = {'City':'Cities'}
dico_mapping_categories = {'Person':'People', 'City':'Cities'}

# Let's extract which entities have the properties that match a WebNLG configuration. The dico will have: { category: { triple_config: [entity1, entity2, etc.] } }
dico_entities_for_triple_configuration = {}
print('Finding which entities have the properties that match a WebNLG configuration...')
for category_label_WebNLG in dico_mapping_categories.keys():
  dico_entities_for_triple_configuration[category_label_WebNLG] = {}
  for triple_config_WebNLG in dico_triple_configs_WebNLG[category_label_WebNLG].keys():
    # Get input configurations (i.e. property sets) extracted from WebNLG
    list_properties_WebNLG = triple_config_WebNLG.split('##')
    # print(list_properties_WebNLG)
    # Get category label used in GREC
    category_label_GREC = dico_mapping_categories[category_label_WebNLG]
    # For each GREC entity, extract the set of properties found on DBpedia
    for entity_name in dico_input_contents_loaded[category_label_GREC].keys():
      # Need a list of strings so we can then convert in sets and compare with other set of property labels
      list_properties_entity = []
      list_triple_objects_entity = dico_input_contents_loaded[category_label_GREC][entity_name]
      for triple_object in list_triple_objects_entity:
        list_properties_entity.append(triple_object.DBprop)
      # Check if any of the WebNLG triple configurations can be built using the properties of each entity
      if set(list_properties_WebNLG).issubset(set(list_properties_entity)):
        if triple_config_WebNLG not in dico_entities_for_triple_configuration[category_label_WebNLG].keys():
          dico_entities_for_triple_configuration[category_label_WebNLG][triple_config_WebNLG] = []
        dico_entities_for_triple_configuration[category_label_WebNLG][triple_config_WebNLG].append(entity_name)

# Save dico_entities_for_triple_configuration as json
with open("dico_entities_for_triple_configuration.json", "w") as outfile:
    json.dump(dico_entities_for_triple_configuration, outfile)

if print_output == True:
  for category_label in dico_entities_for_triple_configuration.keys():
    print('============')
    print(category_label)
    print('============')
    for triple_config_overlap in dico_entities_for_triple_configuration[category_label].keys():
      print('')
      print(triple_config_overlap)
      print('----------------------------')
      for entity_name in dico_entities_for_triple_configuration[category_label][triple_config_overlap]:
        print('-', entity_name)


In [ ]:
#@title Save triple sets in XML format: WebNLG size and input config mirroring
# Here we're trying to build a new dataset that has the same properties as in WebNLG, and the same property configurations in the outputs.
import pickle
import json
from WikipediaPage_Generator.code.utils import create_xml, clear_folder

dico_input_contents = 'GitHub_GREC_NEs'#@param['GitHub_GREC_NEs', 'Made_with_this_notebook']
dico_input_entities = 'GitHub_GREC_NEs'#@param['GitHub_GREC_NEs', 'Made_with_this_notebook']

dico_mapping_categories = {'Person':'People', 'City':'Cities'}

# The dico has the following form: { category: { triple_config: [entity1, entity2, etc.] } }
dico_entities_for_triple_configuration_l = None
if dico_input_entities == 'Made_with_this_notebook':
  dico_entities_for_triple_configuration_l = json.load(open('/content/dico_entities_for_triple_configuration.json', 'r'))
elif dico_input_entities == 'GitHub_GREC_NEs':
  dico_entities_for_triple_configuration_l = json.load(open('/content/Build_KGs_entities/resources/dico_entities_for_triple_configuration_GREC_NEs.json', 'r'))

# dico_input_contents_loaded should be a dico_1 with category names as keys, and a dico_2 as value.
# dico_2 should have entity_name as key, and a list of Triple Objects as value.
# Triple Objects have 3 attributes: DBsubj, DBprop, DBobj
dico_input_contents_loaded = None
if dico_input_contents == 'Made_with_this_notebook':
  with open("dico_input_contents_DBp.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)
elif dico_input_contents == 'GitHub_GREC_NEs':
  with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)

counter_datapoints = 0
# Keep track of how many times an entity is used for an XML, so we can number the inputs corresponding to the same entity (an XML is named after the entity name)
entity_counter = {}
for category_l in dico_entities_for_triple_configuration_l:
  print(category_l)
  # Prepare output folder
  clear_folder(os.path.join(triple2predArg, category_l))
  # if not os.path.exists(os.path.join(triple2predArg, input_category)):
  os.makedirs(os.path.join(triple2predArg, category_l))
  catregory_grec = dico_mapping_categories[category_l]
  for triple_config in dico_entities_for_triple_configuration_l[category_l]:
    print('  ', triple_config, len(dico_entities_for_triple_configuration_l[category_l][triple_config]))
    property_list_l = triple_config.split('##')
    # print(f'{category_l}: {triple_config}: {len(dico_entities_for_triple_configuration_l[category_l][triple_config])}')
    for entity_name in dico_entities_for_triple_configuration_l[category_l][triple_config]:
      # Make filename by using entity name + number of times that entity is being used
      if entity_name not in entity_counter.keys():
        entity_counter[entity_name] = 0
      else:
        entity_counter[entity_name] += 1
      filename = entity_name+'_'+str(entity_counter[entity_name])
      list_triple_objects = dico_input_contents_loaded[catregory_grec][entity_name]
      list_selected_triple_objects = []
      for triple_object in list_triple_objects:
        found_prop = False
        if triple_object.DBprop in property_list_l:
          # print(f'      {entity_name}: found {triple_object.DBprop}.')
          if found_prop == False:
            list_selected_triple_objects.append(triple_object)
            found_prop = True
      # The function that builds an XML expects a list of list IDs that correspond to selected triples. In this context, we want all triples.
      properties_selected = [i for i in range(len(property_list_l))]
      # create xml file passing the entity name to use as filename
      counter_datapoints += 1
      list_triples_text = create_xml(list_selected_triple_objects, properties_selected, category_l, os.path.join(triple2predArg, category_l), entity_name=filename, eid = counter_datapoints)

print(f'----------\n{counter_datapoints} new datapoints were created in total.')

## Build knowledge graphs #2: WebNLG mirror input size distribution only

In [ ]:
#@title Pseudo-code

# Initialize empty dictionary dico_length_ratio_entity

# For each entity in the dataset:
#     Initialize two lists:
#         subject_triples = []  // Triples where the entity is the subject
#         object_triples = []   // Triples where the entity is the object

#     For each triple related to the entity:
#         If entity is the subject:
#             Add triple to subject_triples
#         Else if entity is the object:
#             Add triple to object_triples

#     // Create property-based dictionaries for subjects and objects
#     subject_property_dict = Group subject_triples by property
#     object_property_dict = Group object_triples by property

#     For each desired input length (e.g., 1 to N triples, in our case N=7):
#         For each possible subject/object ratio (e.g., 1:2, 2:1, etc.):
#             // Ensure at least one triple has the entity as subject (Constraint 2)
#             subject_count = number of triples to select as subject
#             object_count = input_length - subject_count

#             possible_subject_triples = []
#             possible_object_triples = []

#             For subject_properties in subject_property_dict:
#                 Randomly select up to 2 triples per property (Constraint 3)
#                 Add to possible_subject_triples

#             For object_properties in object_property_dict:
#                 Randomly select up to 2 triples per property (Constraint 3)
#                 Add to possible_object_triples

#             selected_triples = []
#             Randomly select subject_count triples from possible_subject_triples
#             Add to selected_triples
#             Randomly select object_count triples from possible_object_triples
#             Add to selected_triples

#             Shuffle selected_triples

#             Store selected_triples in dico_length_ratio_entity

# Sample randomly from dico_length_ratio_entity using WebNLG's input length distribution

In [ ]:
#@title Save triple sets in XML format: Implementation of the algorithm for the search+triples sampling
import pickle
import random
import json
from WikipediaPage_Generator.code.utils import create_xml, clear_folder

dico_input_contents = 'GitHub_GREC_NEs'#@param['GitHub_GREC_NEs', 'Made_with_this_notebook']
suffle_selected_triples = True #@param{type:'boolean'}
final_dataset_size = 20 #@param{type:'integer'}
seed = 785 #@param{type:'integer'}
MAX_TRIPLES_SET_LENGTH = 7
MAX_TRIPLES_PER_PROPERTY = 2
DEBUG = False#@param{type:'boolean'}

random.seed(seed)

def group_triples_by_property(triples):
  property_dict = {}
  for triple in triples:
    if triple.DBprop not in property_dict:
      property_dict[triple.DBprop] = []
    property_dict[triple.DBprop].append(triple)
  return property_dict

dico_input_contents_loaded = None
if dico_input_contents == 'Made_with_this_notebook':
  with open("dico_input_contents_DBp.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)
elif dico_input_contents == 'GitHub_GREC_NEs':
  with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)

print(f'Triples for {len(dico_input_contents_loaded["Cities"].keys())+len(dico_input_contents_loaded["People"].keys())} entities were found.')

all_triples_sets = []
for category_name, entities in dico_input_contents_loaded.items():
  for entity_name, triples in entities.items():
    if DEBUG:
      print('\n')
      print(f'Entity name: {entity_name}')
      print('', f'# Triples available: {len(triples)}')
    # Extract all the triples that have the current entity as subject
    subject_triples = [tri for tri in triples if tri.DBsubj == entity_name]
    # Extract all the triples that have the current entity as object
    object_triples = [tri for tri in triples if tri.DBobj == entity_name]
    if DEBUG:
      print('  ', f'# Triples with entity as subject: {len(subject_triples)}')
      print('    ', [f"{o.DBsubj} {o.DBprop} {o.DBobj}" for o in subject_triples[:10]])
      print('  ', f'# Triples with entity as object: {len(object_triples)}')
      print('    ', [f"{o.DBsubj} {o.DBprop} {o.DBobj}" for o in object_triples[:10]])

    # Group subject_triples by property to be able to select randomly a maximum of N triples with the same property (set in MAX_TRIPLES_PER_PROPERTY)
    subject_property_dict = group_triples_by_property(subject_triples)
    # Group object_triples by property
    object_property_dict = group_triples_by_property(object_triples)
    if DEBUG:
      print('  ', f'# Unique properties entity as subj: {len(subject_property_dict)}')
      print('  ', f'# Unique properties entity as obj: {len(object_property_dict)}')

    # create one triple set for each triple set length and ratio
    for triples_len in range(1, MAX_TRIPLES_SET_LENGTH+1):
      if DEBUG:
        print("Triple set length:", triples_len)

      # the subject/object ratio is extracted considering subject count between 1 (possibly change to 0?)
      # (at least one triples with the current entity as subject must be in the triples set)
      # and the triples set length, while object count is triples set length - subject count
      # together they give the subject/object ratio
      for subj_count in range(1, triples_len+1):
        obj_count = triples_len - subj_count
        if DEBUG:
          print(f"  Subject count: {subj_count}, Object count: {obj_count}")

        # extracted the possible triples with the entity as subject
        # for each property, select up to 2 triples, this gives us the complete
        # set of triples with entity as subject from which randomly select the
        # triples for the final triples set
        possible_subject_triples = []
        for prop, prop_triples in subject_property_dict.items():
          possible_subject_triples.extend(random.sample(prop_triples, min(MAX_TRIPLES_PER_PROPERTY, len(prop_triples))))
        # if DEBUG:
        #   print('   ', [f"{o.DBsubj} {o.DBprop} {o.DBobj}" for o in possible_subject_triples])

        # extracted the possible triples with the entity as object
        # for each property, select up to 2 triples, this gives us the complete
        # set of triples with entity as object from which randomly select the
        # triples for the final triples set
        possible_object_triples = []
        for prop, prop_triples in object_property_dict.items():
          possible_object_triples.extend(random.sample(prop_triples, min(MAX_TRIPLES_PER_PROPERTY, len(prop_triples))))

        # check that we have enough triples to select
        if len(possible_subject_triples) >= subj_count and len(possible_object_triples) >= obj_count:
          selected_triples = []
          # select subj_count triples where the entity is subject
          selected_triples.extend(random.sample(possible_subject_triples, subj_count))
          # select obj_count triples where the entity is object
          selected_triples.extend(random.sample(possible_object_triples, obj_count))
          # shuffle the selected triples
          if suffle_selected_triples:
            random.shuffle(selected_triples)
          all_triples_sets.append({
              'triples': selected_triples,
              'subj_count': subj_count,
              'obj_count': obj_count,
              'triples_len': triples_len,
              'entity_name': entity_name,
              'category_name': category_name
          })
          if DEBUG:
            print('   ', [f"{o.DBsubj} {o.DBprop} {o.DBobj}" for o in selected_triples])
        else:
          if DEBUG:
            print('   ', 'Cannot make an input with the subj/obj ratio.')

# sample all_triples_sets to reflect the same triples_len as WebNLG and specific
# TODO replace with automatic extraction of real WebNLG distribution
# IMPORTANT: Express distribution in %
distr = {1: 20.8, 2: 19.6, 3: 19.6, 4: 17.2, 5: 12, 6: 6.4, 7: 4.4}
total_prob = int(sum(distr.values()))
# print(total_prob)
assert total_prob == 100, "Total probability should be 100%"

# If we are missing one sample in the total, find which number to increase by one. E.g. we want 20 samples and get 19 because several numbers are rounded down.
# We look for the number that is the closest to the number above to round it up. e.g. we have 1.2, 3.4 and 3.1 samples, which round to 1, 3 and 3, we want to round 3.4 up to 4.
# Keep here all float numbers to be sampled
num_to_sample_list = [final_dataset_size*value/100 for key, value in distr.items()]
# Keep here all rounded numbers to be sampled
rounded_num_to_sample_list = [round(x) for x in num_to_sample_list]
# If the total of rounded numbers to sample does not equal the actual number to sample, correct the float closest to the above number
position_of_num_to_increase = 0
highest_difference = 0
if sum(rounded_num_to_sample_list) + 1 == final_dataset_size:
  count = 0
  while count < len(num_to_sample_list):
    rounded_number = rounded_num_to_sample_list[count]
    float_number = num_to_sample_list[count]
    difference = float_number - rounded_number
    if difference > highest_difference:
      highest_difference = difference
      position_of_num_to_increase = count
    count += 1
  # Now update the list with rounded numbers to sample
  rounded_num_to_sample_list[position_of_num_to_increase] += 1
  if DEBUG:
    print(num_to_sample_list)
    print(rounded_num_to_sample_list)
    print(f'Position of number to increase: {position_of_num_to_increase} (diff = {highest_difference})')
assert sum(rounded_num_to_sample_list) == final_dataset_size, 'Total number of samples does not match final_dataset_size.'

sampled_triple_sets = []
for count, (triples_len, prob) in enumerate(distr.items()):
  # Select any triple set that has the expected size (allows for multiple triple sets per entity)
  triples_of_len = [tri for tri in all_triples_sets if tri['triples_len'] == triples_len]
  # num_to_sample = round(final_dataset_size * prob / 100)
  num_to_sample = rounded_num_to_sample_list[count]
  if num_to_sample <= len(triples_of_len):
    sampled_triple_sets.extend(random.sample(triples_of_len, num_to_sample))
  else:
    print(f'!!! Could not select triples sets of size {triples_len} (not enough triple sets).')
  print(f'Length: {triples_len}')
  print(f'  # Total triple sets of current size: {len(triples_of_len)}')
  print(f'  # Selected triple sets of current size: {num_to_sample}')
  print(f'  # Total Selected triples at this point: {len(sampled_triple_sets)}')

print(f'# Selected triple sets: {len(sampled_triple_sets)}')
print(sampled_triple_sets[:10])

## Number of triples for each category
# print(f'# Triple sets People: {len([tri for tri in sampled_triples if tri["category_name"]=="People"])}')
# print(f'# Triple sets Cities:{len([tri for tri in sampled_triples if tri["category_name"]=="Cities"])}')
## Total number of triple sets
# print(f'# Triple sets before sampling: {len(all_triples_sets)}')
# print(all_triples_sets[:10])

# Save datapoints in individual XML files
counter_datapoints = 0
entity_counter = {}
folder_name = ''
# print(folder_name)
clear_folder(triple2predArg)
os.makedirs(os.path.join(triple2predArg, 'People'))
os.makedirs(os.path.join(triple2predArg, 'Cities'))
for sampled_triple_set in sampled_triple_sets:
  counter_datapoints += 1
  # print(counter_datapoints)
  list_triple_objects = sampled_triple_set['triples']
  properties_selected = [i for i in range(len(sampled_triple_set['triples']))]
  input_category = sampled_triple_set['category_name']
  folder_name = input_category
  entity_name = sampled_triple_set['entity_name']
  if entity_name not in entity_counter.keys():
    entity_counter[entity_name] = 0
  else:
    entity_counter[entity_name] += 1
  filename = entity_name+'_'+str(entity_counter[entity_name])
  # create xml file passing the entity name to use as filename
  list_triples_text = create_xml(list_triple_objects, properties_selected, input_category, os.path.join(triple2predArg, folder_name), entity_name=filename, eid = counter_datapoints)

# print(sampled_triple_sets[:10])
# TODO save created dataset preprocessing the triples (we cannot save the object as it is)
# for i, tri in enumerate(sampled_triple_sets):
#   tri['triples'] = [f"{o.DBsubj} | {o.DBprop} | {o.DBobj}" for o in tri['triples']]
# with open('/content/sampled_triples.json', 'w') as f:
#   json.dump(sampled_triple_sets, f)

In [ ]:
#@title Check triples extracted from DBpedia
import pickle
import matplotlib.pyplot as plt

# dico_input_contents_loaded should be a dico_1 with category names as keys, and a dico_2 as value.
# dico_2 should have entity_name as key, and a list of Triple Objects as value.
# Triple Objects have 3 attributes: DBsubj, DBprop, DBobj
dico_input_contents_loaded = None
if dico_input_contents == 'Made_with_this_notebook':
  with open("dico_input_contents_DBp.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)
elif dico_input_contents == 'GitHub_GREC_NEs':
  with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)

dico_properties = {}
dico_different_properties = {}
dico_entity_as_subj = {}
dico_entity_as_obj = {}
for category in dico_input_contents_loaded.keys():
  # print(category)
  for entity_name in dico_input_contents_loaded[category].keys():
    all_properties = []
    different_properties = []
    subj_of_properties = []
    obj_of_properties = []
    # print('  ', entity_name)
    for triple_object in dico_input_contents_loaded[category][entity_name]:
      all_properties.append(triple_object.DBprop)
      if triple_object.DBprop not in different_properties:
        different_properties.append(triple_object.DBprop)
      if triple_object.DBsubj == entity_name:
        subj_of_properties.append(triple_object.DBprop)
      if triple_object.DBobj == entity_name:
        obj_of_properties.append(triple_object.DBprop)
    dico_properties[entity_name] = len(all_properties)
    dico_different_properties[entity_name] = len(different_properties)
    dico_entity_as_subj[entity_name] = len(subj_of_properties)
    dico_entity_as_obj[entity_name] = len(obj_of_properties)
    # print(f'    {len(different_properties)} different properties')
    # print(f'    {len(subj_of_properties)} properties with {entity_name} as subject')
    # print(f'    {len(obj_of_properties)} properties with {entity_name} as object')

dico_properties_sorted = {k: v for k, v in sorted(dico_properties.items(), key=lambda item: item[1], reverse=True)}
dico_different_properties_sorted = {k: v for k, v in sorted(dico_different_properties.items(), key=lambda item: item[1], reverse=True)}
dico_entity_as_subj_sorted = {k: v for k, v in sorted(dico_entity_as_subj.items(), key=lambda item: item[1], reverse=True)}
dico_entity_as_obj_sorted = {k: v for k, v in sorted(dico_entity_as_obj.items(), key=lambda item: item[1], reverse=True)}

# Plot dico_properties dictionary using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(dico_properties_sorted.keys(), dico_properties_sorted.values())
plt.xlabel('Entity Name')
plt.ylabel('Number of Properties')
plt.show()

# Plot dico_different_properties dictionary using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(dico_different_properties_sorted.keys(), dico_different_properties_sorted.values())
plt.xlabel('Entity Name')
plt.ylabel('Number of Different Properties')
plt.show()

# Plot dico_entity_as_subj_sorted dictionary using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(dico_entity_as_subj_sorted.keys(), dico_entity_as_subj_sorted.values())
plt.xlabel('Entity Name')
plt.ylabel('Number of Properties with Entity as Subject')
plt.show()

# Plot dico_entity_as_obj_sorted dictionary using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(dico_entity_as_obj_sorted.keys(), dico_entity_as_obj_sorted.values())
plt.xlabel('Entity Name')
plt.ylabel('Number of Properties with Entity as Object')
plt.show()

In [ ]:
#@title Check dataset
from colored import Fore, Back, Style

sizes = {}
property_count_per_datapoint = {}
subjObj_ratios = {}
category_count = {}
for i, striple_set in enumerate(sampled_triple_sets):
  print(f'Datapoint {i+1}:')
  # Check that there is no error with the triple set size
  assert striple_set['subj_count'] + striple_set['obj_count'] == striple_set['triples_len'], 'subj_count + obj_count should be equal to triples_len.'
  print(f'{Fore.green}{Back.white}  subj_count and obj_count match triples_len{Style.reset}')
  assert striple_set['triples_len'] == len(striple_set['triples']), 'triples_len should be equal to the number of triples in the triple set.'
  print(f'{Fore.green}{Back.white}  triples_len and number of triples match{Style.reset}')

  # Collect and check property count for each datapoint
  if str(i) not in property_count_per_datapoint.keys():
    property_count_per_datapoint[str(i)] = {}
  for triple in striple_set['triples']:
    if triple.DBprop not in property_count_per_datapoint[str(i)]:
      property_count_per_datapoint[str(i)][triple.DBprop] = 0
    property_count_per_datapoint[str(i)][triple.DBprop] += 1
  assert property_count_per_datapoint[str(i)][triple.DBprop] <= 2, 'No more than 2 instances of a property per triple set.'
  print(f'{Fore.green}{Back.white}  No more than 2 instances of the same property in the triple set{Style.reset}')

  # Collect all triple sizes
  if striple_set['triples_len'] not in sizes.keys():
    sizes[striple_set['triples_len']] = 0
  sizes[striple_set['triples_len']] += 1

  # Collect category count
  if striple_set['category_name'] not in category_count.keys():
    category_count[striple_set['category_name']] = {}
  if striple_set['entity_name'] not in category_count[striple_set['category_name']].keys():
    category_count[striple_set['category_name']][striple_set['entity_name']] = 0
  category_count[striple_set['category_name']][striple_set['entity_name']] += 1
  # Sort entity names by frequency
  category_count[striple_set['category_name']] = {k: v for k, v in sorted(category_count[striple_set['category_name']].items(), key=lambda item: item[1], reverse=True)}

  # Collect configuration count of triples (i.e. for each size, what is the subj/obj ratio count)
  if striple_set['triples_len'] not in subjObj_ratios.keys():
    subjObj_ratios[striple_set['triples_len']] = {}
  subj_obj_ratio = str(striple_set['subj_count'])+'/'+str(striple_set['obj_count'])
  if subj_obj_ratio not in subjObj_ratios[striple_set['triples_len']].keys():
    subjObj_ratios[striple_set['triples_len']][subj_obj_ratio] = 0
  subjObj_ratios[striple_set['triples_len']][subj_obj_ratio] += 1

print('')
# Check number of datapoints for each size and in total
assert sum(sizes.values()) == final_dataset_size
print(f'{Fore.green}{Back.white}Total number of datapoints: {sum(sizes.values())}{Style.reset}')
print(f'{Fore.green}{Back.white}Triple size distribution: {sizes}{Style.reset}')
print('')
print(f'Subject/Object ratios:    {subjObj_ratios}')
# print sum of people
print(f"Category count:           'People': {sum(category_count['People'].values())} ({len(category_count['People'])} distinct), 'Cities': {sum(category_count['Cities'].values())} ({len(category_count['Cities'])} distinct)")
print(f'People instances:         {category_count["People"]}')
print(f'Cities instances:         {category_count["Cities"]}')

## Build knowledge graphs #3: free distribution

In [ ]:
#@title Optional: examine pickle
# with open("dico_input_contents_DBp.pickle", "rb") as handle:
#   dico_input_contents_loaded = pickle.load(handle)

# print(dico_input_contents_loaded.keys())

In [ ]:
#@title Save triple set in XML format: Large DBpedia dataset
from WikipediaPage_Generator.code.utils import get_first_n_instances_of_props, create_xml, clear_folder
from WikipediaPage_Generator.code.queryDBpediaProps import CheckedTriple, get_resource_types, get_dbo_property_range_or_domain, get_superclasses, sql_query
# from WikipediaPage_Generator.code.utils import create_xml, clear_folder
import json
import pickle
import random

random.seed(42)

# Used for long-input D2T experiments: C:\Users\sfmil\Desktop\ADAPT-2025-2026\MyPapers\2025-06_INLG-longText-D2T\files\triple_sets_full\dico_input_contents_DBp.pickle
dico_input_contents = 'Made_with_this_notebook'#@param['GitHub_GREC_NEs', 'Made_with_this_notebook']
# Map some categories that are found in WebNLG to the name used in WebNLG
dico_mapping_categories = {'People':'Person', 'Cities':'City'}

# Specifies the maximum number of occurrences of each property in the triple set. The lower and upper bounds are used to vary the number of instances for each entity.
 # e.g. one may not always want at most 3 occurrences of the same property with an entity, but rather say between 3 and 6 occurrences so the text looks a bit less mechanical.
max_num_of_instances_of_prop_desired_lowerBound = "3" #@param[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
max_num_of_instances_of_prop_desired_upperBound = "6" #@param[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Specifies the minimun and maximum size of input desired
min_input_size = "8" #@param[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
max_input_size = "199" #@param[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 199, 200, 250, 300, 500, 1000]
# List here properties that cannot have 2 values (to filter bad stuff from i.e. infobox)
# properties_that_can_happen_once_only = ['budget', 'gross', 'imdbId', 'length', 'runtime']
properties_that_can_happen_once_only = json.loads(open('/content/WikipediaPage_Generator/resources/list_props_that_can_happen_once_only.json', 'r').read())
# Select to run the validation of domains and ranges if needed and not done before. Doing it here instead of when collecting triple sets is more efficient be
triple_validation = False #@param{type:"boolean"}
# Either select along with triple_validation, or run on already validated triples.
filter_out_unvalidated_triples = False #@param{type:"boolean"}
# (INLG-2025)
# entities_for_dev = ['Abu_Dhabi', 'Accra', 'Islamabad', 'Lagos', 'Aneurin_Bevan', 'Anthony_Giddens', 'Antoine_Lavoisier', 'Antonio_Negri']
# (ACL 2026)
entities_for_dev = ['Amsterdam', 'Berlin', 'Bucharest', 'Kuala_Lumpur', 'Madrid', 'Moscow', 'Mumbai', 'Singapore', 'Washington,_D.C.', 'Zagreb']

if triple_validation and not filter_out_unvalidated_triples:
  print('You selected triple_validation but not the filtering of triples, so validation will be bypassed because not used.')
# Input data structure should be a dico_1 with category names as keys, and a dico_2 as value.
# dico_2 should have entity_name as key, and a list of Triple Objects as value.
# Triple Objects have 3 attributes: DBsubj, DBprop, DBobj
counter_datapoints = 0
dico_input_contents_loaded = None
if dico_input_contents == 'Made_with_this_notebook':
  with open("dico_input_contents_DBp.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)
elif dico_input_contents == 'GitHub_GREC_NEs':
  with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
    dico_input_contents_loaded = pickle.load(handle)

total_num_selected_props = 0
longest_input_size = 0
for input_category in dico_input_contents_loaded.keys():
  webnlg_input_category = input_category
  if input_category in dico_mapping_categories.keys():
    webnlg_input_category = dico_mapping_categories[input_category]
  print(webnlg_input_category)
  folder_name = f'{webnlg_input_category}_{max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound}'
  clear_folder(os.path.join(triple2predArg, folder_name))
  # if not os.path.exists(os.path.join(triple2predArg, input_category)):
  os.makedirs(os.path.join(triple2predArg, folder_name))
  for entity_name in dico_input_contents_loaded[input_category].keys():
    # Comment this "if" to get v1 data; use "if entity_name in entities_for_dev" to get DEV data.
    if entity_name not in entities_for_dev:
      list_triple_objects = dico_input_contents_loaded[input_category][entity_name]
      # Generate list of indices of all properties that can be part of an input (index in the list of Triple objects that contains all retrieved triples)
      candidate_properties = get_first_n_instances_of_props(list_triple_objects, f'{max_num_of_instances_of_prop_desired_lowerBound}-{max_num_of_instances_of_prop_desired_upperBound}', properties_that_can_happen_once_only)
      print(f'  {entity_name}: pre-selected {len(candidate_properties)}/{len(dico_input_contents_loaded[input_category][entity_name])} properties.')

      filtered_list_triple_objects = []
      # Check whether there is (at least) one intersection between the sets of expected and actual values
      if filter_out_unvalidated_triples:
        # Add triple validation information to candidate triples
        for x in range(len(list_triple_objects)):
          if x in candidate_properties:
            triple_obj = list_triple_objects[x]
            if triple_validation:
              triple_obj.expected_ranges = get_dbo_property_range_or_domain(triple_obj.DBprop, 'range')
              triple_obj.actual_ranges = get_resource_types(triple_obj.DBobj)
              triple_obj.expected_domain = get_dbo_property_range_or_domain(triple_obj.DBprop, 'domain')
              triple_obj.actual_domain = get_resource_types(triple_obj.DBsubj)
            print('    ', x, triple_obj.DBprop)
            # If there is a problem with the range validation, remove triple index from candidates
            if len(triple_obj.expected_ranges) > 0:
              if not set(triple_obj.expected_ranges) & set(triple_obj.actual_ranges):
                print(f'      Range issue: exp. {triple_obj.expected_ranges} - act. {triple_obj.actual_ranges}')
                candidate_properties.remove(x)
            # If there is a problem with the domain validation, remove triple index from candidates
            elif len(triple_obj.expected_domain) > 0:
              if not set(triple_obj.expected_domain) & set(triple_obj.actual_domain):
                print(f'      Domain issue: exp. {triple_obj.expected_domain} - act. {triple_obj.actual_domain}')
                candidate_properties.remove(x)
      else:
        filtered_list_triple_objects = list_triple_objects

      if len(candidate_properties) >= int(min_input_size):
        if len(candidate_properties) > longest_input_size and len(candidate_properties) <= int(max_input_size):
          longest_input_size = len(candidate_properties)
          # print(f'!!! Updated longest input: {longest_input_size} - {entity_name}')
        elif len(candidate_properties) > longest_input_size and len(candidate_properties) > int(max_input_size):
          longest_input_size = int(max_input_size)
          # print(f'!!! Updated longest input: {longest_input_size} - {entity_name}')
        # create xml file passing the entity name to use as filename
        print(f'    {len(candidate_properties[:int(max_input_size)])} included in dataset')
        total_num_selected_props += len(candidate_properties[:int(max_input_size)])
        counter_datapoints += 1
        list_triples_text = create_xml(list_triple_objects, candidate_properties[:int(max_input_size)], webnlg_input_category, os.path.join(triple2predArg, folder_name), entity_name=entity_name, eid = counter_datapoints)

print(f'----------\n{counter_datapoints} new datapoints were created in total (between {int(min_input_size)} and {longest_input_size} triples in input; up to {max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound} instances of the same property in input).')
print(f'Average number of triples per input: {total_num_selected_props/counter_datapoints}')

geography
  Abu_Dhabi: pre-selected 69/527 properties.
    69 included in dataset
  Africa: pre-selected 80/305 properties.
    80 included in dataset
  Alps: pre-selected 32/148 properties.
    32 included in dataset
  Altiplano: pre-selected 8/8 properties.
    8 included in dataset
  American_Samoa: pre-selected 68/256 properties.
    68 included in dataset
  Antarctic_Peninsula: pre-selected 16/57 properties.
    16 included in dataset
  Arabian_Peninsula: pre-selected 37/123 properties.
    37 included in dataset
  Arctic_Circle: pre-selected 2/2 properties.
  Ashgabat: pre-selected 64/315 properties.
    64 included in dataset
  Atacama_Desert: pre-selected 10/17 properties.
    10 included in dataset
  Axel_Heiberg_Island: pre-selected 9/12 properties.
    9 included in dataset
  Baku: pre-selected 106/2123 properties.
    106 included in dataset
  Balkan_Mountains: pre-selected 14/21 properties.
    14 included in dataset
  Bass_Strait: pre-selected 18/162 properties.
    18 in

## Process output XMLs (group, etc.)
Stratified sampling info 100 inputs:
1:20.8 - 2:19.6 - 3:19.6 - 4:17.2 - 5:12 - 6:6.4 - 7:4.4

In [ ]:
#@title Put all XMLs in the same file for sampling
import glob
import codecs
import json

XML_folder = 'XML'#@param['XML', 'XML_Split']

paths_folders_categories = None
out_filename = None
out_entities_list = None

############# In case code is applied to unsplit files generated with the cells above
if XML_folder == 'XML':
  paths_folders_categories = glob.glob('/content/XML/*')
  out_filename = f'/content/XML/D2T-1-FA_same{max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound}_min{min_input_size}_max{max_input_size}.xml'
  out_entities_list = f'/content/XML/list_entities_same{max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound}_min{min_input_size}_max{max_input_size}.json'
############# In case code is applied to split files
elif XML_folder == 'XML_Split':
  paths_folders_categories = glob.glob('/content/XML_Split/*')
  out_filename = f'/content/XML_Split/D2T-1-FA_same3_min8_max{max_num_triples}_SPLIT.xml'
  out_entities_list = f'/content/XML_Split/list_entities_same3_min8_max100_SPLIT-{max_num_triples}.json'

with codecs.open(out_filename, 'w', 'utf-8') as f:
  list_entities = []
  f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
  f.write('<benchmark>\n')
  f.write('  <entries>\n')
  for path_folder_category in sorted(paths_folders_categories):
    list_XMLS_for_category = glob.glob(os.path.join(path_folder_category, '*.xml'))
    for XML in sorted(list_XMLS_for_category):
      filename = os.path.basename(XML).rsplit('.', 1)[0]
      list_entities.append(filename)
      with codecs.open(XML, 'r', 'utf-8') as file:
        XML_lines = file.readlines()
        for line in XML_lines:
          if not line.startswith('<') and not line.startswith('  <'):
            f.write(line)
  f.write('  </entries>\n')
  f.write('</benchmark>\n')
  # Save entity list as json
  # print(list_entities)
  with codecs.open(out_entities_list, 'w', 'utf-8') as f:
    json.dump(list_entities, f)
print('Created XML file!')

Created XML file!


In [ ]:
#@title Zip and download XML folder
from google.colab import files
import os

XML_folder = 'XML'#@param['XML', 'XML_Split']

zip_name_xml = None
folder_to_zip = None
if XML_folder == 'XML':
  zip_name_xml = f'/content/XMLs_{dico_input_contents}_same{max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound}_min{min_input_size}_max{max_input_size}.zip'
  folder_to_zip = '/content/XML'
elif XML_folder == 'XML_Split':
  zip_name_xml = f'v4_XMLs_GREC_NEs_Updated_same{max_num_of_instances_of_prop_desired_lowerBound}to{max_num_of_instances_of_prop_desired_upperBound}_min{min_input_size}_max{max_input_size}_SPLIT-{max_num_triples}.zip'
  folder_to_zip = '/content/XML_Split'

if os.path.exists(zip_name_xml):
  os.remove(zip_name_xml)

! zip -r {zip_name_xml} {folder_to_zip}
files.download(zip_name_xml)

  adding: content/XML/ (stored 0%)
  adding: content/XML/people_3to6/ (stored 0%)
  adding: content/XML/people_3to6/André-Marie_Ampère.xml (deflated 76%)
  adding: content/XML/people_3to6/Maxim_Gorky.xml (deflated 81%)
  adding: content/XML/people_3to6/Darryl_F._Zanuck.xml (deflated 82%)
  adding: content/XML/people_3to6/Washington_Irving.xml (deflated 81%)
  adding: content/XML/people_3to6/Abu_Bakr.xml (deflated 80%)
  adding: content/XML/people_3to6/Leon_Festinger.xml (deflated 73%)
  adding: content/XML/people_3to6/Antonio_Vivaldi.xml (deflated 80%)
  adding: content/XML/people_3to6/Michelangelo_Antonioni.xml (deflated 83%)
  adding: content/XML/people_3to6/Henry_Miller.xml (deflated 75%)
  adding: content/XML/people_3to6/Lorenzo_de_Medici.xml (deflated 83%)
  adding: content/XML/people_3to6/Vladimir_Vysotsky.xml (deflated 83%)
  adding: content/XML/people_3to6/Graham_Greene.xml (deflated 82%)
  adding: content/XML/people_3to6/Rafael_Nadal.xml (deflated 80%)
  adding: content/XML/pe

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Split each triple set in an XML file into several smaller triple sets
from WikipediaPage_Generator.code.utils import TripleSet, Triple_withID, create_xml, clear_folder, balanced_split_with_max, extract_info_from_WebNLG_XML, sort_WebNLG_XMLs, split_XMLs
import os

##### INPUT VARIABLES
path_xml = "/content/v4_long-inputs_GREC_same3_min8_max100.xml"#@param{type:"string"}
path_DBprops_count = "/content/WikipediaPage_Generator/resources/dico_count_occurrences_dbp_props.json"#@param{type:"string"}
# Take into account a few triples can be added to a group in case boundaries between groups are changed (to keep same properties in the same group)
max_num_triples = 22#@param{type:"integer"}
path_save_XMLs = "/content/XML_Split"#@param{type:"string"}
debug = False#@param{type:"boolean"}

if os.path.exists(path_xml):
  split_XMLs(path_xml, path_DBprops_count, max_num_triples, path_save_XMLs, DEBUG = debug)

In [ ]:
#@title Compare Semantic accuracy pickle file and Long-input D2T pickle file
import pickle
import json

dico_input_contents_SemEx_loaded = None
with open("/content/Build_KGs_entities/resources/dico_input_contents_DBp_GREC_NEs.pickle", "rb") as handle:
  dico_input_contents_SemEx_loaded = pickle.load(handle)

dico_input_contents_LongIn_loaded = None
with open("/content/dico_input_contents_DBp.pickle", "rb") as handle:
  dico_input_contents_LongIn_loaded = pickle.load(handle)

def showContentsDico(dico_input_contents_loaded):
  dico_contents = {}
  for category in dico_input_contents_loaded.keys():
    print(category)
    if category not in dico_contents.keys():
      dico_contents[category] = {}
    for entity_name in dico_input_contents_loaded[category].keys():
      print(f'  {entity_name} ({len(dico_input_contents_loaded[category][entity_name])} properties)')
      if entity_name not in dico_contents[category].keys():
        dico_contents[category][entity_name] = []
      for triple_object in dico_input_contents_loaded[category][entity_name]:
        # print('    ', triple_object.DBsubj, triple_object.DBprop, triple_object.DBobj)
        triple = f'{triple_object.DBsubj} {triple_object.DBprop} {triple_object.DBobj}'
        dico_contents[category][entity_name].append(triple)
  return dico_contents

dico_SemEx = showContentsDico(dico_input_contents_SemEx_loaded)
dico_LongIn = showContentsDico(dico_input_contents_LongIn_loaded)

# Save dicos as jsons
with open('dico_input_contents_DBp_GREC_NEs_SemEx.json', 'w') as f:
  json.dump(dico_SemEx, f)
with open('dico_input_contents_DBp_GREC_NEs_LongIn.json', 'w') as f:
  json.dump(dico_LongIn, f)

# More stuff

In [ ]:
#@title Test Functions

# To split large XML inputs that can trigger memory issues in FORGe
# 1 Check what the most frequent entity is (as subject or object)
# 2 order all triples in which this entity is subject by frequency of occurrence of the property in DBpedia
# 3 order all triples in which this entity is object by frequency of occurrence of the property in DBpedia
  # If the entity is the object of a property that ends with a preposition, put it at the bottom in the list.
# 4 Repeat 2 and 3 for the next most frequent entities as subject.
# 5 Create an XML with at most n entities. Do not cut between two same properties.

##### INSTALLS
# ! pip install xmltodict
##### IMPORTS
from WikipediaPage_Generator.code.utils import create_xml, clear_folder
import xmltodict
import json
import codecs
import os
from colored import Fore, Back, Style
##### INPUT VARIABLES
# path_xml = "/content/v3_long-inputs_GREC_same3_min8_max100.xml"
path_xml = "/content/v3_long-inputs_GREC_same3_min8_just1.xml"
path_DBprops_count = "/content/WikipediaPage_Generator/resources/dico_count_occurrences_dbp_props.json"
# Take into account a few triples can be added to a group in case boundaries between groups are changed (to keep same properties in the same group)
max_num_triples = 22
path_save_XMLs = "/content/XML_Split"
debug = False

##### FUNCTIONS
class TripleSet:
  def __init__(self, triples_list, category, eid, shape, shape_type):
    self.triples = triples_list
    self.category = category
    self.eid = eid
    self.size = len(triples_list)
    self.shape = shape
    self.shape_type = shape_type
    self.entities_by_frequency = []
    # The main entity is the one that has the most occurrences as subject or object (if several have the same num of occurrences, "max" returns the first one)
    entity_counter_dico = {}
    for triple in self.triples:
      if triple.DBsubj not in entity_counter_dico.keys():
        entity_counter_dico[triple.DBsubj] = 1
      else:
        entity_counter_dico[triple.DBsubj] += 1
      if triple.DBobj not in entity_counter_dico.keys():
        entity_counter_dico[triple.DBobj] = 1
      else:
        entity_counter_dico[triple.DBobj] += 1
    self.entities_by_frequency = sorted(entity_counter_dico, key=entity_counter_dico.get, reverse=True)
    # print(entity_counter_dico)
    # print(self.entities_by_frequency)

class Triple_withID:
  def __init__(self, prop, subj_value, obj_value, triple_id):
    self.DBprop = prop
    self.DBsubj = subj_value
    self.DBobj = obj_value
    self.id = triple_id

def balanced_split_with_max(N1, N2):
  """
  Function that splits as evenly as possible a number N1 into smaller numbers each as close as possible to another number N2 without being larger than N2.
  For example, if N1 == 50 and N2 == 20, the output is 17, 17 16.
  """
  assert N1 >= N2, "N1 must be greater than or equal to N2"
  assert N2 >= 1, "N1 must be at least 1"
  # Start with the minimal number of parts needed to respect the max constraint
  for k in range(N1 // N2, N1 + 1):
    # print(f'k: {k}')
    base = N1 // k
    remainder = N1 % k
    # print(f'N1//N2, N1+1: {N1//N2}, {N1+1}')
    # print(f'base: {base}')
    # print(f'remainder: {remainder}')
    # print()
    # The largest part will be base + 1 (if remainder > 0)
    if base + (1 if remainder > 0 else 0) <= N2:
      result = [base + 1] * remainder + [base] * (k - remainder)
      return result

def extract_info_from_WebNLG_XML (path_input_XML):
  """
  path_input_XML: Path to an XML file that contains triple sets, e.g. as provided in the WebNLG shared tasks.
  returns a list of TripleSet objects. Each object contains as attributes: triples (a list of Triple objects), category, eid, size, shape, main_entity
  """
  with codecs.open(path_input_XML, 'r', 'utf-8') as file:
    XML_file = file.read()
    XML_dict = xmltodict.parse(XML_file)
    print(f'    Reading file {path_input_XML}..')
    # triple_sets_list will be a list of objects of class TripleSet
    total_number_of_triples = 0
    triple_sets_list = []
    if isinstance(XML_dict['benchmark']['entries']['entry'], list):
      print(f"      There are {len(XML_dict['benchmark']['entries']['entry'])} inputs in the original XML file.")
      for entry in XML_dict['benchmark']['entries']['entry']:
        category = entry['@category']
        eid = entry['@eid']
        size = entry['@size']
        shape = entry['@shape']
        shape_type = entry['@shape-type']
        # mtriples_list will be a list of objects of class Triple
        mtriples_list = []
        # Get modified triples
        if isinstance(entry['modifiedtripleset']['mtriple'], list):
          for triple_id, mtriple in enumerate(entry['modifiedtripleset']['mtriple']):
            triple_object = Triple_withID(mtriple.split(' | ')[1], mtriple.split(' | ')[0], mtriple.split(' | ')[2], triple_id)
            mtriples_list.append(triple_object)
        else:
          mtriples_list.append(entry['modifiedtripleset']['mtriple'])
        assert int(size) == len(mtriples_list), f'Error: found size {size} but found {len(mtriples_list)} triples.'
        total_number_of_triples += len(mtriples_list)
        # Create object of class TripleSet
        tripleSet_object = TripleSet(mtriples_list, category, eid, shape, shape_type)
        triple_sets_list.append(tripleSet_object)
    else:
      print(f"      There is 1 input in the original XML file.")
      category = XML_dict['benchmark']['entries']['entry']['@category']
      eid = XML_dict['benchmark']['entries']['entry']['@eid']
      size = XML_dict['benchmark']['entries']['entry']['@size']
      shape = XML_dict['benchmark']['entries']['entry']['@shape']
      shape_type = XML_dict['benchmark']['entries']['entry']['@shape-type']
      # Block repeated from above
      # mtriples_list will be a list of objects of class Triple
      mtriples_list = []
      # Get modified triples
      if isinstance(XML_dict['benchmark']['entries']['entry']['modifiedtripleset']['mtriple'], list):
        for triple_id, mtriple in enumerate(XML_dict['benchmark']['entries']['entry']['modifiedtripleset']['mtriple']):
          triple_object = Triple_withID(mtriple.split(' | ')[1], mtriple.split(' | ')[0], mtriple.split(' | ')[2], triple_id)
          mtriples_list.append(triple_object)
      else:
        mtriples_list.append(XML_dict['benchmark']['entries']['entry']['modifiedtripleset']['mtriple'])
      assert int(size) == len(mtriples_list), f'Error: found size {size} but found {len(mtriples_list)} triples.'
      total_number_of_triples += len(mtriples_list)
      # Create object of class TripleSet
      tripleSet_object = TripleSet(mtriples_list, category, eid, shape, shape_type)
      triple_sets_list.append(tripleSet_object)

    print(f"      There are {total_number_of_triples} input triples in the original XML file.")

  return triple_sets_list

def sort_WebNLG_XMLs (path_input_XML, path_DBprops_count):
  """
  path_input_XML: Path to an XML file that contains triple sets, e.g. as provided in the WebNLG shared tasks. The code expects that all triples mention the same entity, as subject or object.
  path_DBprops_count: Path to a json file that contains DBpedia properties as keys (e.g. "http://dbpedia.org/ontology/birthPlace") and number of occurrences on DBpedia as values (e.g 1486579).
  This function returns a list of TripleSets objects. TripleSet.triples contains Triple objects; in each triple set, Triple objects are sorted by "importance" (i.e. sorted by frequency of entity in the triple set, and by frequency of property on DBpedia)
  """
  print('  Sorting triples sets by frequency of entity in the triple set, and by frequency of respective properties on DBpedia...')
  dico_count_occurrences_dbp_props = json.loads(codecs.open(path_DBprops_count, 'r', 'utf-8').read())
  triple_sets_list = extract_info_from_WebNLG_XML (path_input_XML)
  new_triple_set_Objects_list = []
  total_number_of_triples = 0
  for triple_set in triple_sets_list:
    # print(triple_set.eid, triple_set.category, triple_set.size, triple_set.entities_by_frequency[0])
    # Make a list where we will store the order of the triples using their index in the triple_set list
    # E.g. list_triple_indices = [0, 4, 5, 2, 3, 1]
    list_triple_indices = []
    # Process entities by their respective importance in the triple set, so the most frequently found entity will go first, the second most frequently found will go second, and so on.
    for entity_name in triple_set.entities_by_frequency:
      # print(f'  {entity_name}')
      # Make a list of property labels with the http://dbpedia.org/ontology/ prefix, one with the properties where the entity is subject, and one with the properties where the entity is object
      # The properties in the ..._Subj list will go first, the properties in the ..._Obj list will go after.
      list_dico_count_occurrences_dbp_props_keys_Subj = [[f'http://dbpedia.org/ontology/{triple.DBprop}', triple.id] for triple in triple_set.triples if triple.DBsubj == entity_name]
      list_dico_count_occurrences_dbp_props_keys_Obj = [[f'http://dbpedia.org/ontology/{triple.DBprop}', triple.id] for triple in triple_set.triples if triple.DBobj == entity_name]
      # Order that list according to the count in path_DBprops_count
      sorted_list_dico_count_occurrences_dbp_props_keys_Subj = sorted(list_dico_count_occurrences_dbp_props_keys_Subj, key=lambda x: dico_count_occurrences_dbp_props[x[0]], reverse=True)
      sorted_list_dico_count_occurrences_dbp_props_keys_Obj = sorted(list_dico_count_occurrences_dbp_props_keys_Obj, key=lambda x: dico_count_occurrences_dbp_props[x[0]], reverse=True)
      # print(f'    {sorted_list_dico_count_occurrences_dbp_props_keys_Subj}')
      # print(f'    {sorted_list_dico_count_occurrences_dbp_props_keys_Obj}')
      # Now put all the triple indices for the current entity in list_triple_indices, starting with the triples in which the entity is subject
      for list_triple_indices_Subj in sorted_list_dico_count_occurrences_dbp_props_keys_Subj:
        # To avoid duplicated triples:
        if list_triple_indices_Subj[1] not in list_triple_indices:
          list_triple_indices.append(list_triple_indices_Subj[1])
      for list_triple_indices_Obj in sorted_list_dico_count_occurrences_dbp_props_keys_Obj:
        if list_triple_indices_Obj[1] not in list_triple_indices:
          list_triple_indices.append(list_triple_indices_Obj[1])

    #Now add the triples in a list, ordering the triples as defined in list_triple_indices (the create_xml function expects the triples ordered already)
    new_triples_list = [triple_set.triples[i] for i in list_triple_indices]
    assert len(new_triples_list) == len(triple_set.triples), f'Expected {len(triple_set.triples)} triples, found {len(new_triples_list)}'
    total_number_of_triples += len(new_triples_list)
    # print(len(new_triples_list), [new_triples_list[x].id for x in range(len(new_triples_list))])
    new_triple_set_Objects_list.append(TripleSet(new_triples_list, triple_set.category, triple_set.eid, triple_set.shape, triple_set.shape_type))
  assert len(new_triple_set_Objects_list) == len(triple_sets_list), f'Expected {len(triple_sets_list)} triple sets, found {len(new_triple_set_Objects_list)}'
  print(f'    There are {len(new_triple_set_Objects_list)} sorted triple sets...')
  print(f'    There are {total_number_of_triples} input triples in the sorted XML file.')

  return new_triple_set_Objects_list

def split_XMLs (path_input_XML, path_DBprops_count, max_num_triples, path_save_XMLs, DEBUG = False):
  """
  path_input_XML: Path to an XML file that contains triple sets, e.g. as provided in the WebNLG shared tasks. The code expects that all triples mention the same entity, as subject or object.
  path_DBprops_count: Path to a json file that contains DBpedia properties as keys (e.g. "http://dbpedia.org/ontology/birthPlace") and number of occurrences on DBpedia as values (e.g 1486579).
  max_num_triples: the maximum number of triples desired in an XML
  path_save_XMLs: the path where the output XMLs should be created
  This function creates individual XML files for each split triple set.
  """
  print('Splitting XML file...')
  clear_folder(path_save_XMLs)
  os.makedirs(path_save_XMLs)
  # Get the list of TripleSet objects with the triples re-ordered. The object contains the following:
  # self.triples, self.category, self.eid, self.size, self.shape, self.shape_type, self.entities_by_frequency
  new_triple_set_Objects_list = sort_WebNLG_XMLs(path_input_XML, path_DBprops_count)
  total_number_of_XMLs = 0
  total_number_of_triples = 0
  for new_triple_set in new_triple_set_Objects_list:
    if DEBUG:
      print(new_triple_set.size, new_triple_set.entities_by_frequency[0])
    # Get "ideal" triple set split (see balanced_split_with_max function)
    even_slices = None
    if new_triple_set.size > max_num_triples:
      # balanced_split_with_max returns a sequence of numbers that stand for a number of properties.
      groups = balanced_split_with_max(new_triple_set.size, max_num_triples)
      # Let's convert that to a sequence of numbers that correspond to list slices: [10, 10, 5] becomes [10, 20, 25]
      even_slices = [sum(groups[:i]) for i in range(len(groups)+1)]
    else:
      even_slices = [0] + [new_triple_set.size]
    if DEBUG:
      print(f'  Before: {even_slices}')

    # Initialise new list
    new_slices = [0]
    # Now we need to check if the split happened between two occurrences of the same property, which we'd like to avoid
    # even_slices has at least 2 numbers, 0 and the end of the first or only slice.
    if len(even_slices) > 2:
      # Check for intermediate group boundaries (i.e. exclude the first boundary, which is 0, and the last one, because there is no property after it)
      for boundary in even_slices[1:-1]:
        previous_same_property = 0
        # Since in the way even_slices is built, the last slices are the smallest ones, it's better to move boundaries to the left.
        while new_triple_set.triples[boundary+previous_same_property].DBprop == new_triple_set.triples[boundary+previous_same_property-1].DBprop:
          previous_same_property -= 1
        if previous_same_property < 0:
          new_slices.append(boundary+previous_same_property)
          if DEBUG:
            print(f'  {Fore.red}{Back.yellow}!!! Changed split {boundary}, {previous_same_property}!{Style.reset}')
        else:
          new_slices.append(boundary)
      # Add last boundary
      new_slices.append(even_slices[-1])
    else:
      # Add second and last boundary
      new_slices.append(even_slices[1])
    if DEBUG:
      print(f'  After: {new_slices}')

    # Create XMLs
    # Set parameters for calling function that creates XMLs
    input_category = new_triple_set.category
    folder_name = input_category+'_max'+str(max_num_triples)
    entity_name = new_triple_set.entities_by_frequency[0]
    eid = new_triple_set.eid
    # Clear/Create output folder
    if not os.path.exists(os.path.join(path_save_XMLs, folder_name)):
      os.makedirs(os.path.join(path_save_XMLs, folder_name))

    # For each slice of the triple set, create an XML file
    count_files_created = 0
    for count_files, i in enumerate(range(len(new_slices)-1)):
      list_triple_objects = new_triple_set.triples[new_slices[i]:new_slices[i+1]]
      properties_selected_by_user = [i for i in range(len(list_triple_objects))] # Use all properties
      unique_entity_name = entity_name+'_'+str(count_files)
      list_triples_text = create_xml(list_triple_objects, properties_selected_by_user, input_category, os.path.join(path_save_XMLs, folder_name), entity_name=unique_entity_name, eid = eid)
      count_files_created += 1
      total_number_of_triples += len(list_triple_objects)
    total_number_of_XMLs += count_files_created

  print(f'  Created {total_number_of_XMLs} split XML files of approximate size {max_num_triples}.')
  print(f'  There are {total_number_of_triples} input triples in the split XML files.')

split_XMLs(path_xml, path_DBprops_count, max_num_triples, path_save_XMLs, DEBUG = debug)

In [ ]:
#@title Regroup FORGe outputs for split long-input inputs.
# Above we needed to split some XMLs because FORGe can explode on inputs too large, so now we need to build on output file aligned with the input file
import codecs
import os
import glob
import json

# 1 - Create a folder FORGe and upload outputs there, e.g. all_EN_dev_out_aligned.txt or v4_long-inputs_GREC_same3_min8_max100_SPLIT-22_en_000-299__SMorphText.conll_out.txt
# 2 - Upload file list_entities_same3_min8_max100_SPLIT-22.json created at the same time as the split XML (and found in the same folder).

forge_out_files = glob.glob('/content/FORGe/*.txt')
list_entities_split = json.load(codecs.open('/content/list_entities_same3_min8_max100_SPLIT-22.json', 'r', 'utf-8'))

# Put all FORGe texts in a list
all_forge_lines = []
for forge_out_file in sorted(forge_out_files):
  with codecs.open(forge_out_file, 'r', 'utf-8') as f:
    forge_out_lines = f.readlines()
    all_forge_lines += forge_out_lines

current_entity = None
count_different_entities = 0
with codecs.open('FORGe-all.txt', 'w', 'utf-8') as f:
  for line_count, entity_with_count in enumerate(list_entities_split):
    entity_name = entity_with_count.rsplit('_', 1)[0]
    print(f'{line_count} - {entity_name}')
    line = ''
    if entity_name != current_entity:
      current_entity = entity_name
      if line_count == 0:
        line = all_forge_lines[line_count].strip()
      else:
        line = line + '\n'
        line = line + all_forge_lines[line_count].strip()
      count_different_entities += 1
    else:
      line = line + ' '+all_forge_lines[line_count].strip()
    f.write(line)

print(f'Found {count_different_entities} different entities.')

In [ ]:
#@title Make a shorter version of the reference texts
import os
import glob
import codecs

lang = 'EN'#@param['EN', 'GA']
proportion_kept = 0.9#@param{type:"slider", min:0, max:1, step:0.01}
folder_path = f'8_{lang}'
zip_path = f"/content/{folder_path}.zip"
# 9 is made with proportion_kept = 0.9
# 10 is made with proportion_kept = 0.7
# 11 is made with proportion kept = 0.5
folder_out_num = '9'
folder_out = f'{folder_out_num}_{lang}'

if not os.path.exists(os.path.join('/content', folder_out)):
  os.makedirs(os.path.join('/content', folder_out))

#Unzip file
# ! rm -r {folder_path}
if not os.path.exists(folder_path):
  ! unzip {zip_path} -d /content/{folder_path}


for text_file_path in glob.glob(f'/content/{folder_path}/8/*.txt'):
  filename = text_file_path.split('/')[-1]
  new_filename = filename.replace('[8_', '['+folder_out_num+'_')
  print(new_filename)
  sentences_list = codecs.open(text_file_path, 'r', 'utf-8').read().strip().split('. ')
  print(f'  {len(sentences_list)} sentences found.')
  num_sentences_kept = int(round(len(sentences_list)*proportion_kept, 0))
  print(f'  {len(sentences_list[:num_sentences_kept])} sentences kept')
  with codecs.open(os.path.join('/content', folder_out, new_filename), 'w', 'utf-8') as f:
    for sentence in sentences_list[:num_sentences_kept]:
      f.write(sentence+'. ')

# zip and download folder_out
! zip -r /content/{folder_out}.zip /content/{folder_out}
from google.colab import files
files.download(f'/content/{folder_out}.zip')

## Create sheets for human evaluation

In [1]:
#@title Create fake output files for testing the code below
# import os
# import codecs

# if not os.path.exists('/content/normalised_outputs'):
#   os.makedirs('/content/normalised_outputs')

# # Create fake outputs for 3 systems
# for i in range(3):
#   with codecs.open(f'/content/normalised_outputs/{i}-outputs-en.txt', 'w', 'utf-8') as f:
#     for j in range(669):
#       f.write(f'sys_{i}-line_{j} (sys and line counts started at 0). English text here')
#       if j < 668:
#         f.write('\n')

In [59]:
#@title Unzip the real files for the code below
import os

# Files from C:\Users\sfmil\...\data\random_900_entities\outputs\LLMs_normalised_outputs\content\normalised_outputs
language = 'en'#@param['en', 'ga']
path_zip = '/content/outputs_D2T-1-FA_'+language+'.zip'

if os.path.exists('/content/normalised_outputs'):
  ! rm -r /content/normalised_outputs

# Unzip files
! unzip {path_zip} -d /content/normalised_outputs

Archive:  /content/outputs_D2T-1-FA_en.zip
  inflating: /content/normalised_outputs/0-e2e_D2T-1-FA_en.txt  
  inflating: /content/normalised_outputs/1-default_D2T-1-FA_en.txt  
  inflating: /content/normalised_outputs/2-unified_D2T-1-FA_en.txt  


In [60]:
#@title Load sampling info and output texts (upload D2T-1-FA_samplingData.csv first)
# We want to map the D2T-1-FA_samplingData.csv file to a format that matches dico_annotator_assignments created above
import os
import codecs
import csv
import glob
import sys
from pandas import *
import pandas as pd

# CSV from C:\Users\sfmil\...\data\random_900_entities\evaluation
# Upload the sampling data csv into the repo
path_sampling_data_csv = '/content/D2T-1-FA_samplingData.csv'
path_output_files = '/content/normalised_outputs'

# Extract sampling info
# Original_id matches the original ID in the XML file (starts at 1); The "id" starts at 0, so it will match the line number in the output files.
dico_sampled_ids_per_size = {}
head, tail = os.path.split(path_sampling_data_csv)
df_triple_sets = pd.read_csv(path_sampling_data_csv)
# Extract all different values in the strat_field column:
strat_field_labels = sorted(df_triple_sets['strat_field'].unique())
# Create a dictionary that maps strat_field_labels to integers starting from 0
strat_field_labels_to_int = {i : strat_field_labels[i] for i in range(len(strat_field_labels))}
print(f'Mapping of strat_field values to the following keys: {strat_field_labels_to_int}')
# strat_field_labels_to_int looks like this: {0: np.int64(0), 1: np.int64(2), 2: np.int64(4), 3: np.int64(6), 4: np.int64(8), 5: np.int64(10), 6: np.int64(12), 7: np.int64(14), 8: np.int64(16), 9: np.int64(18)}
# Now for each "normalised" size, get the text IDs
for n_int in strat_field_labels_to_int.keys():
  dico_sampled_ids_per_size[n_int] = [df_triple_sets.loc[df_triple_sets['strat_field'] == strat_field_labels_to_int[n_int], 'id'].tolist()][0]
print('\ndico_sampled_ids_per_size: normalised size ranges (starting from 0) and the corresponding sampled text IDs (starting from 0):')
print('', dico_sampled_ids_per_size)

# Extract output texts
dico_output_texts = {}
for outfile in glob.glob(os.path.join(path_output_files, '*.txt')):
  system_id = int(outfile.split('/')[-1].split('-')[0])
  dico_output_texts[system_id] = {}
  with codecs.open(outfile, 'r', 'utf-8') as f:
    lines_output = f.readlines()
    for i, line_output in enumerate(lines_output):
      dico_output_texts[system_id][i] = line_output.strip()
print('+\ndico_output_texts: system IDs (starting from 0), all text IDs (starting from 0) and corresponding texts:')
print('', dico_output_texts)
if 0 not in dico_output_texts.keys():
  sys.exit('There should be one system output file with the prefix "0_"!')

# At this point, dico_sampled_ids_per_size looks like this, where keys are size categories:
# {0: [570, 88, 246, 485, 371, 665], 1: [], 2: [213, 456, 348, 418, 382, 424], 3: [], 4: [652, 64, 332, 642, 318, 172], 5: [], 6: [479, 0, 303, 614, 7, 73]... }
# And dico_output_texts looks like this, where keys are system IDs, and next level keys are input IDs:
# {'3': {0: 'sys_3-line_0 (line count started at 0). English text here', 1: 'sys_3-line_1 (line count started at 0). English text here', ...} }

# Let's now build a dictionary with all and only the contents for the spreadsheet
dico_for_spreadsheets = {}
for system_id in dico_output_texts.keys():
  # Create on key per system ID in the new dico
  dico_for_spreadsheets[system_id] = {}
  for size in dico_sampled_ids_per_size.keys():
    # For each system ID, create a key for each size
    dico_for_spreadsheets[system_id][size] = []
    # Now go get the corresponding text for each systemID+textID. The texts will be in the same order for all systems since the text ID come from a unique list of IDs in dico_sampled_ids_per_size.
    for text_id in dico_sampled_ids_per_size[size]:
      dico_for_spreadsheets[system_id][size].append(dico_output_texts[system_id][text_id])
print('=\ndico_for_spreadsheets: system IDs (starting from 0), normalised size ranges (starting from 0) and the corresponding sampled texts')
print('', dico_for_spreadsheets)

Mapping of strat_field values to the following keys: {0: np.int64(0), 1: np.int64(2), 2: np.int64(4), 3: np.int64(6), 4: np.int64(8), 5: np.int64(10), 6: np.int64(12), 7: np.int64(14), 8: np.int64(16), 9: np.int64(18)}

dico_sampled_ids_per_size: normalised size ranges (starting from 0) and the corresponding sampled text IDs (starting from 0):
 {0: [570, 88, 246], 1: [213, 456, 348], 2: [652, 64, 332], 3: [479, 0, 303], 4: [25, 137, 141], 5: [147, 52, 91], 6: [126, 165, 74], 7: [117, 124, 13], 8: [76, 68, 34], 9: [144, 53, 140]}
+
dico_output_texts: system IDs (starting from 0), all text IDs (starting from 0) and corresponding texts:
 {2: {0: 'Abu Dhabi is both the capital city and a major metropolis of the United Arab Emirates, situated within the Emirate of Abu Dhabi. Covering a total area of 972.0 square kilometers and located at an elevation of 27.0 meters, the city has a population of approximately 1,570,000. Governed as a municipality, Abu Dhabi is administered by the Abu Dhabi C

In [61]:
#@title Automatically assign texts to evaluators (latin square)
# To distribute evaluators as evenly as possible across input sizes and systems.
# With two annotators per text.

from collections import defaultdict
import csv
import math

# Use dico_for_spreadsheets to get the number of systems.
num_systems = len(dico_for_spreadsheets.keys())
# Get the number of different sizes using the first key in dico_for_spreadsheets
num_sizes = len(dico_for_spreadsheets[list(dico_for_spreadsheets.keys())[0]])

# IMPORTANT: make sure the number of output per sizes matches the provided data.
# It should be len(dico_sampled_ids_per_size[0]), but I leave it manual here so it is possible to play around with the numbers.
# If a smaller number than len(dico_sampled_ids_per_size[0]) is chosen, the largest data sizes will be dropped.
num_outputs_per_size = 3  #@param{type:'slider', min:1, max:10}
num_annotators = 6        #@param{type:'slider', min:1, max:10}

def choose_offset(num_annotators: int) -> int:
  """ Function to compute the offset we will use to optimise the latin square"""
  # Hand-pick for some small N
  #
  # The “+2” choice for 6 annotators is just a clean way to guarantee two different annotators per item while keeping things symmetric and balanced across the 6 annotators; it is not the only possible shift, but it is a convenient one given there are 6 annotators.
  # With +1 for the two annotators of the same items, the distribution is not as even, e.g. each output is seen by 4 instead of 5 (when using +2) different annotators. With +3, there would be only 3 different pairs of annotators, so worse distribution of annotators overall.
  if num_annotators <= 4:
    return 1
  elif num_annotators > 4 and num_annotators <= 6:
    return 2
  elif num_annotators == 7 or num_annotators == 8:
    return 3
  elif num_annotators == 9 or num_annotators == 10:
    return 4
  else:
    # Randomly decided by me
    estimate = int(num_annotators/(num_systems-1))
    return estimate

offset = choose_offset(num_annotators)
print(f'Chose offset = {offset} for {num_annotators} annotators.')

def twoAnnotators_per_item(size_i, system_j, output_k, offset):
  # We want a1 and a2 to have a difference of "offset" (see choose_offset function) to maximise evaluator distribution. Using the "modulo" operator to ensure we always have a valid evaluator number.
  a1 = (size_i + system_j + output_k) % num_annotators # add "+ 1" at the end if we want to start annotator ID at 1 instead of 0
  a2 = (size_i + system_j + output_k + offset) % num_annotators # add "+ 1" at the end if we want to start annotator ID at 1 instead of 0
  # print(f'  size: {size_i} system: {system_j} output: {output_k} annotators {a1}, {a2}')
  return a1, a2

# Collect assignments per annotator
# defaultdict(list) automatically provides an empty list [] for any new key accessed.
dico_annotator_assignments = defaultdict(list)
for i in range(num_sizes):        # size index
  for j in range(num_systems):    # system index
    for k in range(num_outputs_per_size):  # output index
      a1, a2 = twoAnnotators_per_item(i, j, k, offset)
      item = {"size": i, "system": j, "output": k}
      dico_annotator_assignments[a1].append(item)
      dico_annotator_assignments[a2].append(item)

# Sanity check
for annotator in sorted(dico_annotator_assignments.keys()):
  # print(f"Annotator {annotator}: {len(dico_annotator_assignments[annotator])} items. {dico_annotator_assignments[annotator]}")
  print(f"Annotator {annotator}: {len(dico_annotator_assignments[annotator])} items.")

# Build and print one table: system × global_output_id → list of annotators. Each cell will typically contain 2 annotators (since we want 2 per text).
table = {s: defaultdict(list) for s in range(num_systems)}
for size in range(num_sizes):
  for system in range(num_systems):
    for k in range(num_outputs_per_size):
      a1, a2 = twoAnnotators_per_item(size, system, k, offset)
      global_id = size * num_outputs_per_size + k
      table[system][global_id].extend([a1, a2])
systems = range(num_systems)
all_cols = range(num_sizes * num_outputs_per_size)
header = ["IDs"] + [str(c) for c in all_cols]
print('\n6/size: Size 0: 0-5; Size 1: 6-11; Size 2: 12-17; Size 3: 18-23; Size 4: 24-29; Size 5: 30-35; Size 6: 36-41; Size 7: 42-47; Size 8: 48-53; Size 9: 54-59')
print('3/size: Size 0: 0-2; Size 1: 3-5; Size 2: 6-8; Size 3: 9-11; Size 4: 12-14; Size 5: 15-17; Size 6: 18-20; Size 7: 21-23; Size 8: 24-26; Size 9: 27-29')
print('Rows = Systems, Columns = Outputs\n---------------------------------\n'+"\t".join(header))
for s in systems:
  row = [f"S{s}"]
  for gid in all_cols:
    anns = table[s].get(gid, [])
    cell = ",".join(str(a) for a in sorted(set(anns))) if anns else ""
    row.append(cell)
  print("\t".join(row))

# Write one CSV per annotator
for a in range(num_annotators):
  filename = f"annotator_{a}.csv"
  with open(filename, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["size", "system", "output"])
    writer.writeheader()
    for row in dico_annotator_assignments[a]:
      writer.writerow(row)

Chose offset = 2 for 6 annotators.
Annotator 0: 30 items.
Annotator 1: 28 items.
Annotator 2: 28 items.
Annotator 3: 30 items.
Annotator 4: 32 items.
Annotator 5: 32 items.

6/size: Size 0: 0-5; Size 1: 6-11; Size 2: 12-17; Size 3: 18-23; Size 4: 24-29; Size 5: 30-35; Size 6: 36-41; Size 7: 42-47; Size 8: 48-53; Size 9: 54-59
3/size: Size 0: 0-2; Size 1: 3-5; Size 2: 6-8; Size 3: 9-11; Size 4: 12-14; Size 5: 15-17; Size 6: 18-20; Size 7: 21-23; Size 8: 24-26; Size 9: 27-29
Rows = Systems, Columns = Outputs
---------------------------------
IDs	0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29
S0	0,2	1,3	2,4	1,3	2,4	3,5	2,4	3,5	0,4	3,5	0,4	1,5	0,4	1,5	0,2	1,5	0,2	1,3	0,2	1,3	2,4	1,3	2,4	3,5	2,4	3,5	0,4	3,5	0,4	1,5
S1	1,3	2,4	3,5	2,4	3,5	0,4	3,5	0,4	1,5	0,4	1,5	0,2	1,5	0,2	1,3	0,2	1,3	2,4	1,3	2,4	3,5	2,4	3,5	0,4	3,5	0,4	1,5	0,4	1,5	0,2
S2	2,4	3,5	0,4	3,5	0,4	1,5	0,4	1,5	0,2	1,5	0,2	1,3	0,2	1,3	2,4	1,3	2,4	3,5	2,4	3,5	0,4	3,5	0,4	1,5	0,4	1,5	0,2	1,5	0,2	1,3


In [62]:
#@title Print dictionaries built so far
print('We now have three useful dictionaries!')
print('\ndico_sampled_ids_per_size (contains IDs of sampled texts): normalised size ranges (starting from 0) and the corresponding sampled text IDs (starting from 0):')
print('', dico_sampled_ids_per_size)
print('\ndico_for_spreadsheets (contains sampled texts sorted by size and system): system IDs (starting from 0), normalised size ranges (starting from 0) and the corresponding sampled texts:')
print('', dico_for_spreadsheets)
print('\ndico_annotator_assignments (contains assignments as combination of input size, system ID and local output ID): annotator IDs (starting from 0), lists of dicos that contain {system IDs, normalised size ranges and normalised output IDs} (all starting from 0)')
# Local on the line above means the position of an output in the list of sampled outputs for a certain size, so if 6 outputs per size were selected above, this relative ID will be a number between 0 and 5.
print('', dico_annotator_assignments)

We now have three useful dictionaries!

dico_sampled_ids_per_size (contains IDs of sampled texts): normalised size ranges (starting from 0) and the corresponding sampled text IDs (starting from 0):
 {0: [570, 88, 246], 1: [213, 456, 348], 2: [652, 64, 332], 3: [479, 0, 303], 4: [25, 137, 141], 5: [147, 52, 91], 6: [126, 165, 74], 7: [117, 124, 13], 8: [76, 68, 34], 9: [144, 53, 140]}

dico_for_spreadsheets (contains sampled texts sorted by size and system): system IDs (starting from 0), normalised size ranges (starting from 0) and the corresponding sampled texts:
 {2: {0: ['Paul Gauguin, whose birth name was Eugène Henri Paul Gauguin, was born on June 7, 1848, and died on May 8, 1903, in the Marquesas Islands. He was the author of several notable works, including "The Painter of Sunflowers," "Ia Orana Maria," "Arearea no varua ino," "Still Life with Profile of Laval," and "Parau na te varua ino." In addition to his paintings, Gauguin\'s artwork was featured as the cover art for "The St

In [63]:
#@title Authenticate in Sheets and open working sheet (upload credentials first)
import gspread
from google.oauth2.service_account import Credentials

# Credentials from C:\Users\sfmil\Desktop\ADAPT-2025-2026\Admin\Google

# 1. Auth scopes and credentials
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
# Upload credentials file to the repo
SERVICE_ACCOUNT_FILE = "/content/credentials.json"

creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES,
)

# 2. Authorize client
client = gspread.authorize(creds)

In [64]:
#@title Fill up annotator sheets (first assign sheet IDs manually in dico_annotator_sheets below)
# I can probably use the position of the texts in dico_for_spreadsheets[systemID][sizeID] to match with the 'output' key in dico_annotator_assignments.
# Because for all systems, the list of output IDs comes from the same source list, so the output IDs are in the same order for each system.
import random
from googleapiclient.errors import HttpError
import time
import re

# For long D2T eval, used 42
random.seed(42)

# The first annotator has ID = 0
dico_annotator_sheets = None
if language == 'en':
  dico_annotator_sheets = {0:'1nfuDA-1RizII_FxCWqDLRXfgb6DG9b5bUJCIA96a5ig',
                           1:'1ij9Y5ydxb5mL6TwsvubOGVzVdjGi8aUP26mb7Id1DGQ',
                           2:'1zVLDqm69-9Gwb2GOCEWh_svb98axId3-oX3ONUF3aUE',
                           3:'12vuBNdzaLUWoZo63XhKY6aEZXOMMGhuG6tM1oV3aUnw',
                           4:'1hGh0L9Hca4NLnqLknZph-ctf5I13nImkVfYeGIZZtWU',
                           5:'14EWDERWpiHaIscwkGsFlsqpSFyX_-5nnwsyhe_ukmRg'}
elif language == 'ga':
  dico_annotator_sheets = {0:'1nRr-twKJXE4oJlRNUslL6sgdGXWwwwDbbjBG9pMgZus',
                           1:'1p8y_-wdZM-M7gQtBtCn915YIfdDe8CN3zkW32O9N-Dk',
                           2:'1kekPQXrt--pnzUz2ejYmbSi2ZldbfI0lSZJV5-Y8UXE',
                           3:'1nnfmXA_GcM0bDpobzaMuR9v-MEqBZOPCHdQa4LOebrA',
                           4:'1ihzKym3K-SRZ7yypBMktIFDWUTqMTRUzQILztLI5fQ0',
                           5:'1BfW-Sa8BorYbU7MHXNCmT6ZYfqYd90GXkdVRDnr7hCs'}
if 0 not in dico_annotator_sheets.keys():
  sys.exit('There should be one key 0 in the dictionary!')

for annotator_id in dico_annotator_sheets.keys():
  # 3. Open spreadsheet and first worksheet
  spreadsheet = client.open_by_key(dico_annotator_sheets[annotator_id])
  worksheet = spreadsheet.get_worksheet(0)  # first sheet

  data_columns_AB = []
  data_column_G = []
  # Note for self: I need to use list() here to actually create a copy of dico_annotator_assignments[annotator_id] before shuffling, otherwise the random seed doesn't work.
  # list_dicos_assignments = dico_annotator_assignments[annotator_id] does not create a new list—it creates a reference (like a pointer) to the exact same list object already stored in the dictionary.
  list_dicos_assignments = list(dico_annotator_assignments[annotator_id])
  # Shuffle the assignments otherwise they are sorted by size, which is not ideal for the evaluation (each text would be larger than the previous one)
  random.shuffle(list_dicos_assignments)
  print(list_dicos_assignments)
  for i, dico_assignment in enumerate(list_dicos_assignments):
    # Get info about each text assignment for the current annotator
    system_ID = dico_assignment['system']
    size_ID = dico_assignment['size']
    output_relID = dico_assignment['output']
    # Get full ID from the dico_sampled_ids_per_size dico
    output_fullID = dico_sampled_ids_per_size[size_ID][output_relID]
    print(f'  {i} - sys:{system_ID} - size:{size_ID} - idRel:{output_relID} - idFull:{output_fullID}')
    # Get text
    eval_text = dico_for_spreadsheets[system_ID][size_ID][output_relID]
    # Add linebreaks after each sentence.
    eval_text_linebreaks = re.subn(r'([^\s][^\s]\.) ([A-Z])', r'\g<1>\n\g<2>', eval_text)[0]

    # 4. Write content to cells; too many calls per minute need to batch the calls
    # worksheet.update_acell("A"+str(i+7), i)
    # worksheet.update_acell("B"+str(i+7), eval_text)
    # worksheet.update_acell("G"+str(i+7), f'{output_fullID}-{system_ID}-{size_ID}-{output_relID}')
    data_columns_AB.append([i, eval_text_linebreaks])
    data_column_G.append([f'{output_fullID}-{system_ID}-{size_ID}-{output_relID}'])

  # print(data_columns_AB)
  worksheet.update(data_columns_AB, 'A7')
  # print(data_column_G)
  worksheet.update(data_column_G, 'G7')

[{'size': 6, 'system': 2, 'output': 2}, {'size': 4, 'system': 1, 'output': 1}, {'size': 3, 'system': 1, 'output': 2}, {'size': 9, 'system': 0, 'output': 1}, {'size': 8, 'system': 0, 'output': 2}, {'size': 2, 'system': 2, 'output': 0}, {'size': 2, 'system': 1, 'output': 1}, {'size': 4, 'system': 0, 'output': 0}, {'size': 3, 'system': 2, 'output': 1}, {'size': 4, 'system': 2, 'output': 0}, {'size': 3, 'system': 1, 'output': 0}, {'size': 8, 'system': 2, 'output': 2}, {'size': 9, 'system': 2, 'output': 1}, {'size': 7, 'system': 2, 'output': 1}, {'size': 5, 'system': 0, 'output': 1}, {'size': 9, 'system': 1, 'output': 0}, {'size': 0, 'system': 2, 'output': 2}, {'size': 4, 'system': 0, 'output': 2}, {'size': 6, 'system': 0, 'output': 0}, {'size': 1, 'system': 1, 'output': 2}, {'size': 5, 'system': 1, 'output': 0}, {'size': 9, 'system': 1, 'output': 2}, {'size': 2, 'system': 0, 'output': 2}, {'size': 8, 'system': 2, 'output': 0}, {'size': 2, 'system': 2, 'output': 2}, {'size': 3, 'system': 0,

## Process human evaluation sheets

In [48]:
#@title Extract ratings from spreadsheets (upload CSVs in human-evals folder)
import glob
import os
import pandas as pd
import json

#======PARAMS=======
path_eval_folder = '/content/human-evals'
language = 'ga'#@param['en', 'ga']
# first_score_row counter starts at 0
annot_id_separator_in_filename = '_'+language+'-'
file_extension = 'csv'
# In the pilot, the rating values were slightly different, and there was no ID info in the sheets
pilot = True#@param{type:'boolean'}
allowed_scores = {'Very good: There are no grammatical errors in the text.':5,
                  'Very good: The text flows well and can be read easily.':5,
                  'Very good: The text is well organised and coherent.': 5,
                  'Good: Somewhere between Very good and Fair.':4,
                  'Fair: There are a few grammatical errors in the text.':3,
                  'Fair: The text has occasional disfluencies.':3,
                  'Fair: The text could be better organised for a better coherence.':3,
                  'Poor: Somewhere between Very poor and Fair.':2,
                  'Very poor: There are numerous grammatical errors in the text.':1,
                  'Very poor: The text does not flow well and I had to start over to understand some parts.':1,
                  'Very poor: The text is poorly organised and lacks coherence.':1
}
if pilot == True:
  allowed_scores['Good: There are no grammatical errors in the text.'] = 5
  allowed_scores['Good: The texts flows well and can be read easily.'] = 5
  allowed_scores['Good: The text is well organised and coherent.'] = 5
  allowed_scores['Poor: There are numerous grammatical errors in the text.'] = 1
  allowed_scores['Poor: The text does not flow well and I had to start over to understand some parts.'] = 1
  allowed_scores['Poor: The text is poorly organised and lacks coherence.'] = 1
#======PARAMS=======

def check_scores(score_list, allowed_scores):
  for score in score_list:
    # print(score, allowed_scores)
    assert score in allowed_scores.keys(), f'Score out of range: {score}'
    numerical_score = allowed_scores[score]
    return numerical_score

paths_files = glob.glob(os.path.join(path_eval_folder, '*.'+file_extension))

list_human_eval_datapoints = []
for path_file in paths_files:
  annot_id = path_file.rsplit(annot_id_separator_in_filename, 1)[1].rsplit('.', 1)[0]
  print('Annotator:', annot_id)
  df = pd.read_csv(path_file)
  # Make a copy of df considering the 6th row as the header row
  # 1. Set new headers from the 5th row (index 4)
  df.columns = df.iloc[4]
  # 2. Drop the first 5 rows (0 to 5) and reset the index
  df_noInstructions = df.iloc[5:].reset_index(drop=True)
  # print(df_noInstructions)
  header_values = list(df.columns.values)
  # print(header_values)

  x = 0
  while x < len(df_noInstructions):
    dico_eval_datapoint = {}
    gram_score = df_noInstructions.at[x, 'Grammaticality']
    flu_score = df_noInstructions.at[x, 'Fluency']
    coh_score = df_noInstructions.at[x, 'Coherence']
    # Check score and get numerical value
    gram_score_num = check_scores([gram_score], allowed_scores)
    flu_score_num = check_scores([flu_score], allowed_scores)
    coh_score_num = check_scores([coh_score], allowed_scores)
    # ID is in column 'G', which has no header because it's kind of hidden (not present in pilot)
    id = None
    if pilot == True:
      id = str(x)+'_0'
    else:
      system_ID = df_noInstructions.iloc[x, 6].split('-')[1].split('-')[0]
      input_id = df_noInstructions.iloc[x, 6].split('-')[0]
      id = str(input_id)+'_'+str(system_ID)
    print(f'  {x} id={id} - gram: {gram_score_num}; flu: {flu_score_num}; coh: {coh_score_num}')
    dico_eval_datapoint['id'] = id
    dico_eval_datapoint['input'] = ''
    dico_eval_datapoint['output'] = ''
    dico_eval_datapoint['grammaticality'] = gram_score_num
    dico_eval_datapoint['fluency'] = flu_score_num
    dico_eval_datapoint['coherence'] = coh_score_num
    dico_eval_datapoint['annotator_id'] = int(annot_id)
    list_human_eval_datapoints.append(dico_eval_datapoint)
    x += 1

# Save list_human_eval_datapoints as json file
with open(os.path.join('/content', language+'_annotations.json'), 'w') as f:
  json.dump(list_human_eval_datapoints, f)

Annotator: 4
  0 id=0_0 - gram: 3; flu: 3; coh: 3
  1 id=1_0 - gram: 3; flu: 5; coh: 3
  2 id=2_0 - gram: 5; flu: 5; coh: 5
  3 id=3_0 - gram: 3; flu: 5; coh: 3
Annotator: 2
  0 id=0_0 - gram: 5; flu: 5; coh: 5
  1 id=1_0 - gram: 3; flu: 5; coh: 5
  2 id=2_0 - gram: 3; flu: 3; coh: 3
  3 id=3_0 - gram: 5; flu: 5; coh: 5
Annotator: 1
  0 id=0_0 - gram: 5; flu: 3; coh: 5
  1 id=1_0 - gram: 5; flu: 3; coh: 5
  2 id=2_0 - gram: 5; flu: 3; coh: 5
  3 id=3_0 - gram: 5; flu: 3; coh: 5
Annotator: 3
  0 id=0_0 - gram: 3; flu: 5; coh: 5
  1 id=1_0 - gram: 5; flu: 3; coh: 3
  2 id=2_0 - gram: 3; flu: 3; coh: 3
  3 id=3_0 - gram: 3; flu: 3; coh: 3
Annotator: 0
  0 id=0_0 - gram: 3; flu: 3; coh: 3
  1 id=1_0 - gram: 3; flu: 3; coh: 5
  2 id=2_0 - gram: 3; flu: 5; coh: 3
  3 id=3_0 - gram: 3; flu: 5; coh: 3


In [45]:
#@title Calculate Fleiss's Kappa
import pandas as pd
import json
import os

path_json = '/content/ga_annotations.json'#@param
# num_agg_categories can be 2 or 3 for now (into how many categories should the 7 scores aggregated)
num_agg_categories = 2#@param{'type':'integer'}

'''
Created on Aug 1, 2016
@author: skarumbaiah

Computes Fleiss' Kappa
Joseph L. Fleiss, Measuring Nominal Scale Agreement Among Many Raters, 1971.
'''

def checkInput(rate, n):
    """
    Check correctness of the input matrix
    @param rate - ratings matrix
    @return n - number of raters
    @throws AssertionError
    """
    N = len(rate)
    k = len(rate[0])
    assert all(len(rate[i]) == k for i in range(N)), "Row length != #categories)"
    assert all(isinstance(rate[i][j], int) for i in range(N) for j in range(k)), "Element not integer"
    assert all(sum(row) == n for row in rate), "Sum of ratings != #raters)"

def fleissKappaPerplexity(rate, n):
    N = len(rate)
    k = len(rate[0])
    print(f"#raters = {n}, #subjects = {N}, #categories = {k}")
    checkInput(rate, n)

    PA = sum((sum(x*x for x in row) - n) / (n * (n - 1)) for row in rate) / N
    print("PA =", PA)

    pj = [sum(row[j] for row in rate) / (N * n) for j in range(k)]
    PE = sum(p**2 for p in pj)
    print("PE =", PE)

    try:
        kappa = (PA - PE) / (1 - PE)
        kappa = float(f"{kappa:.3f}")
    except ZeroDivisionError:
        print("Expected agreement = 1")
        kappa = float("nan")

    print("Fleiss' Kappa =", kappa)
    return kappa

def fleissKappa(rate,n):
    """
    Computes the Kappa value
    @param rate - ratings matrix containing number of ratings for each subject per category
    [size - N X k where N = #subjects and k = #categories]
    @param n - number of raters
    @return fleiss' kappa
    """

    N = len(rate)
    k = len(rate[0])
    print("#raters (n) = ", n, ", #subjects (N) = ", N, ", #categories (k) = ", k)
    checkInput(rate, n)

    #mean of the extent to which raters agree for the ith subject
    PA = sum([(sum([i**2 for i in row])- n) / (n * (n - 1)) for row in rate])/N
    print("PA = ", PA)

    # mean of squares of proportion of all assignments which were to jth category
    PE = sum([j**2 for j in [sum([rows[i] for rows in rate])/(N*n) for i in range(k)]])
    print("PE =", PE)

    kappa = -float("inf")
    try:
        kappa = (PA - PE) / (1 - PE)
        kappa = float("{:.3f}".format(kappa))
    except ZeroDivisionError:
        print("Expected agreement = 1")

    print("Fleiss' Kappa =", kappa)

    return kappa

# Code above expects a matrix of d rows and s columns where:
# d is the number of data points (also called "subjects")
# s is the number of categories, i.e. possible scores (in our case, for one system, that's 7: 1 to 7).
# Then at each position in the matrix, there should be the number n of annotators who chose the score s for datapoint d.
# rate = [
#     [2,0,0,0],
#     [0,2,0,0],
#     [0,0,2,0],
#     [0,0,0,2]
# ]
# kappa = fleissKappa(rate,2)
# assert(kappa==1)

def count_ratings_exact(ratings_list):
  # Create a list with 5 zeros, each index representing counts for 1-5
  counts = [0] * 5
  for rating in ratings_list:
    if 1 <= rating <= 5:
      counts[rating - 1] += 1
    else:
      raise ValueError(f"Invalid rating: {rating}. Ratings must be between 1 and 5.")
  return counts

def count_ratings_aggregate(ratings_list, num_categories):
  # Create a list with 3 zeros, each index representing counts for 1-7
  counts = [0] * num_categories
  for rating in ratings_list:
    # print(f'rating: {rating}')
    if num_categories == 2:
      if rating == 1 or rating == 2:
        counts[0] += 1
      elif rating == 3 or rating == 4 or rating == 5:
        counts[1] += 1
    if num_categories == 3:
      if rating == 1 or rating == 2:
        counts[0] += 1
      elif rating == 3:
        counts[1] += 1
      elif rating == 4 or rating == 5:
        counts[2] += 1
      else:
        raise ValueError(f"Invalid rating: {rating}. Ratings must be between 1 and 5.")
  # print(counts)
  return counts

def makeFleissMatrix(json_filepath, num_categories):
  # for IAA with all 5 scores
  scores_matrix_dico_5scores = {'coherence':[], 'fluency':[], 'grammaticality':[]}
  # fro IAA with 3 scores, aggregating 1-2 and 4-5
  scores_matrix_dico_Aggscores = {'coherence':[], 'fluency':[], 'grammaticality':[]}
  num_annot = 0
  with open(json_filepath) as f:
    json_data = json.load(f)
  df_scores = pd.read_json(json_filepath)
  df_scores_per_item = df_scores.groupby(['id'])
  # key is the full ID
  for key, item in df_scores_per_item:
    # print(key)
    coh_scores = []
    gram_scores = []
    flu_scores = []
    # Collect scores
    for index, row in item.iterrows():
      coh_scores.append(row['coherence'])
      gram_scores.append(row['grammaticality'])
      flu_scores.append(row['fluency'])
    assert len(coh_scores) == len(gram_scores) == len(flu_scores)
    if num_annot == 0:
      num_annot = len(coh_scores)
    assert len(coh_scores) == num_annot
    # Convert list of scores into a list of count of each score, e.g. [3, 3, 4, 2, 5] -> [0, 1, 2, 1, 1, 0, 0] (7 scores) or  [3, 3, 4, 2, 5] -> [3, 2] (2 scores)
    scores_matrix_dico_5scores['coherence'].append(count_ratings_exact(coh_scores))
    scores_matrix_dico_5scores['grammaticality'].append(count_ratings_exact(gram_scores))
    scores_matrix_dico_5scores['fluency'].append(count_ratings_exact(flu_scores))
    scores_matrix_dico_Aggscores['coherence'].append(count_ratings_aggregate(coh_scores, num_categories))
    scores_matrix_dico_Aggscores['grammaticality'].append(count_ratings_aggregate(gram_scores, num_categories))
    scores_matrix_dico_Aggscores['fluency'].append(count_ratings_aggregate(flu_scores, num_categories))
  return scores_matrix_dico_5scores, scores_matrix_dico_Aggscores, num_annot

# How many categories do we want for the aggregated scores?
scores_matrix_dico_5scores, scores_matrix_dico_Aggscores, num_annot = makeFleissMatrix(path_json, num_agg_categories)

print(scores_matrix_dico_5scores)
print('Fleiss with 5 scores\n----------------')
for qualityCriterion in scores_matrix_dico_5scores:
  print(qualityCriterion)
  fleissKappa(scores_matrix_dico_5scores[qualityCriterion], num_annot)

print(scores_matrix_dico_Aggscores)
print('\nFleiss with aggregated scores\n----------------')
for qualityCriterion in scores_matrix_dico_Aggscores:
  print(qualityCriterion)
  fleissKappa(scores_matrix_dico_Aggscores[qualityCriterion], num_annot)

{'coherence': [[0, 0, 2, 0, 3], [0, 0, 2, 0, 3], [0, 0, 3, 0, 2], [0, 0, 3, 0, 2]], 'fluency': [[0, 0, 3, 0, 2], [0, 0, 3, 0, 2], [0, 0, 3, 0, 2], [0, 0, 2, 0, 3]], 'grammaticality': [[0, 0, 3, 0, 2], [0, 0, 3, 0, 2], [0, 0, 3, 0, 2], [0, 0, 3, 0, 2]]}
Fleiss with 5 scores
----------------
coherence
#raters (n) =  5 , #subjects (N) =  4 , #categories (k) =  5
PA =  0.4
PE = 0.5
Fleiss' Kappa = -0.2
fluency
#raters (n) =  5 , #subjects (N) =  4 , #categories (k) =  5
PA =  0.4
PE = 0.5050000000000001
Fleiss' Kappa = -0.212
grammaticality
#raters (n) =  5 , #subjects (N) =  4 , #categories (k) =  5
PA =  0.4
PE = 0.52
Fleiss' Kappa = -0.25
{'coherence': [[0, 5], [0, 5], [0, 5], [0, 5]], 'fluency': [[0, 5], [0, 5], [0, 5], [0, 5]], 'grammaticality': [[0, 5], [0, 5], [0, 5], [0, 5]]}

Fleiss with aggregated scores
----------------
coherence
#raters (n) =  5 , #subjects (N) =  4 , #categories (k) =  2
PA =  1.0
PE = 1.0
Expected agreement = 1
Fleiss' Kappa = -inf
fluency
#raters (n) =  5 , 